### Train the inductive link prediction model

In [1]:
data_name = 'WN18RR_v4'
model_id = 'SiaLP_3_new'
lower_bound = 1
upper_bound_path = 10
upper_bound_subg = 3

In [2]:
#difine the names for saving
model_name = 'Model_' + model_id + '_' + data_name
one_hop_model_name = 'One_hop_model_' + model_id + '_' + data_name
ids_name = 'IDs_' + model_id + '_' + data_name

In [3]:
import librosa
import opensmile
import os
import sys
import numpy as np
import random
import pickle

from collections import defaultdict
from copy import deepcopy
from sklearn.utils import shuffle
from sys import getsizeof

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import initializers
from tensorflow.keras.utils import plot_model

In [4]:
class LoadKG:
    
    def __init__(self):
        
        self.x = 'Hello'
        
    def load_train_data(self, data_path, one_hop, data, s_t_r, entity2id, id2entity,
                     relation2id, id2relation):
        
        data_ = set()
    
        ####load the train, valid and test set##########
        with open (data_path, 'r') as f:
            
            data_ini = f.readlines()
                        
            for i in range(len(data_ini)):
            
                x = data_ini[i].split()
                
                x_ = tuple(x)
                
                data_.add(x_)
        
        ####relation dict#################
        index = len(relation2id)
     
        for key in data_:
            
            if key[1] not in relation2id:
                
                relation = key[1]
                
                relation2id[relation] = index
                
                id2relation[index] = relation
                
                index += 1
                
                #the inverse relation
                iv_r = '_inverse_' + relation
                
                relation2id[iv_r] = index
                
                id2relation[index] = iv_r
                
                index += 1
        
        #get the id of the inverse relation, by above definition, initial relation has 
        #always even id, while inverse relation has always odd id.
        def inverse_r(r):
            
            if r % 2 == 0: #initial relation
                
                iv_r = r + 1
            
            else: #inverse relation
                
                iv_r = r - 1
            
            return(iv_r)
        
        ####entity dict###################
        index = len(entity2id)
        
        for key in data_:
            
            source, target = key[0], key[2]
            
            if source not in entity2id:
                                
                entity2id[source] = index
                
                id2entity[index] = source
                
                index += 1
            
            if target not in entity2id:
                
                entity2id[target] = index
                
                id2entity[index] = target
                
                index += 1
                
        #create the set of triples using id instead of string        
        for ele in data_:
            
            s = entity2id[ele[0]]
            
            r = relation2id[ele[1]]
            
            t = entity2id[ele[2]]
            
            if (s,r,t) not in data:
                
                data.add((s,r,t))
            
            s_t_r[(s,t)].add(r)
            
            if s not in one_hop:
                
                one_hop[s] = set()
            
            one_hop[s].add((r,t))
            
            if t not in one_hop:
                
                one_hop[t] = set()
            
            r_inv = inverse_r(r)
            
            s_t_r[(t,s)].add(r_inv)
            
            one_hop[t].add((r_inv,s))
            
        #change each set in one_hop to list
        for e in one_hop:
            
            one_hop[e] = list(one_hop[e])

In [5]:
class ObtainPathsByDynamicProgramming:

    def __init__(self, amount_bd=50, size_bd=50, threshold=20000):
        
        self.amount_bd = amount_bd #how many Tuples we choose in one_hop[node] for next recursion
                        
        self.size_bd = size_bd #size bound limit the number of paths to a target entity t
        
        #number of times paths with specific length been performed for recursion
        self.threshold = threshold
        
    '''
    Given an entity s, the function will find the paths from s to other entities, using recursion.
    
    One may refer to LeetCode Problem 797 for details:
        https://leetcode.com/problems/all-paths-from-source-to-target/
    '''
    def obtain_paths(self, mode, s, t_input, lower_bd, upper_bd, one_hop):

        if type(lower_bd) != type(1) or lower_bd < 1:
            
            raise TypeError("!!! invalid lower bound setting, must >= 1 !!!")
            
        if type(upper_bd) != type(1) or upper_bd < 1:
            
            raise TypeError("!!! invalid upper bound setting, must >= 1 !!!")
            
        if lower_bd > upper_bd:
            
            raise TypeError("!!! lower bound must not exced upper bound !!!")
            
        if s not in one_hop:
            
            raise ValueError('!!! entity not in one_hop. Please work on existing entities')

        #here is the result dict. Its key is each entity t sharing paths from s
        #The value of each t is a set containing the paths from s to t
        #These paths can be either the direct connection r, or a multi-hop path
        res = defaultdict(set)
        
        #qualified_t contains the types of t we want to consider,
        #that is, what t will be added to the result set.
        qualified_t = set()

        #under this mode, we will only consider the direct neighbour of s
        if mode == 'direct_neighbour':
        
            for Tuple in one_hop[s]:
            
                t = Tuple[1]
                
                qualified_t.add(t)
        
        #under this mode, we will only consider one specified entity t
        elif mode == 'target_specified':
            
            qualified_t.add(t_input)
        
        #under this mode, we will consider any entity
        elif mode == 'any_target':
            
            for s_any in one_hop:
                
                qualified_t.add(s_any)
                
        else:
            
            raise ValueError('not a valid mode')
        
        '''
        We use recursion to find the paths
        On current node with the path [r1, ..., rk] and on-path entities {s, e1, ..., ek-1, node}
        from s to this node, we will further find the direct neighbor t' of this node. 
        If t' is not an on-path entity (not among s, e1,...ek-1, node), we recursively proceed to t' 
        '''
        def helper(node, path, on_path_en, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict):

            #when the current path is within lower_bd and upper_bd, 
            #and the node is among the qualified t, and it has not been fill of paths w.r.t size_limit,
            #we will add this path to the node
            if (len(path) >= lower_bd) and (len(path) <= upper_bd) and (
                node in qualified_t) and (len(res[node]) < self.size_bd):
                
                res[node].add(tuple(path))
                    
            #won't start new recursions if the current path length already reaches upper limit
            #or the number of recursions performed on this length has reached the limit
            if (len(path) < upper_bd) and (count_dict[len(path)] <= self.threshold):
                                
                #temp list is the id list for us to go-over one_hop[node]
                temp_list = [i for i in range(len(one_hop[node]))]
                random.shuffle(temp_list) #so we random-shuffle the list
                
                #only take 20 recursions if there are too many (r,t)
                for i in temp_list[:self.amount_bd]:
                    
                    #obtain tuple of (r,t)
                    Tuple = one_hop[node][i]
                    r, t = Tuple[0], Tuple[1]
                    
                    #add to count_dict even if eventually this step not proceed
                    count_dict[len(path)] += 1
                    
                    #if t not on the path and we not exceed the computation threshold, 
                    #then finally proceed to next recursion
                    if (t not in on_path_en) and (count_dict[len(path)] <= self.threshold):

                        helper(t, path + [r], on_path_en.union({t}), res, qualified_t, 
                               lower_bd, upper_bd, one_hop, count_dict)

        length_dict = defaultdict(int)
        count_dict = defaultdict(int)
        
        helper(s, [], {s}, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict)
        
        return(res, count_dict)

In [6]:
train_path = '../data/' + data_name + '/train.txt'
valid_path = '../data/' + data_name + '/valid.txt'
test_path = '../data/' + data_name + '/test.txt'

In [7]:
#load the classes
Class_1 = LoadKG()
Class_2 = ObtainPathsByDynamicProgramming()

In [8]:
#define the dictionaries and sets for load KG
one_hop = dict() 
data = set()
s_t_r = defaultdict(set)

#define the dictionaries, which is shared by initail and inductive train/valid/test
entity2id = dict()
id2entity = dict()
relation2id = dict()
id2relation = dict()

#fill in the sets and dicts
Class_1.load_train_data(train_path, one_hop, data, s_t_r,
                        entity2id, id2entity, relation2id, id2relation)

In [9]:
#define the dictionaries and sets for load KG
one_hop_valid = dict() 
data_valid = set()
s_t_r_valid = defaultdict(set)

#fill in the sets and dicts
Class_1.load_train_data(valid_path, one_hop_valid, data_valid, s_t_r_valid,
                        entity2id, id2entity, relation2id, id2relation)

In [10]:
#define the dictionaries and sets for load KG
one_hop_test = dict() 
data_test = set()
s_t_r_test = defaultdict(set)

#fill in the sets and dicts
Class_1.load_train_data(test_path, one_hop_test, data_test, s_t_r_test,
                        entity2id, id2entity, relation2id, id2relation)

#### Build the path-based siamese neural network structure

We use biLSTM to train on the input path embedding sequence to predict the output embedding or the relation.

In [11]:
# Input layer, using integer to represent each relation type
#note that inputs_path is the path inputs, while inputs_out_re is the output relation inputs
fst_path = keras.Input(shape=(None,), dtype="int32")
scd_path = keras.Input(shape=(None,), dtype="int32")
thd_path = keras.Input(shape=(None,), dtype="int32")

#the relation input layer (for output embedding)
id_rela = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 300-dimensional vector as input,
# note that we add another "space holder" embedding, 
# which hold the spaces if the initial length of paths are not the same
in_embd_var = layers.Embedding(len(relation2id)+1, 300)

# Obtain the embedding
fst_p_embd = in_embd_var(fst_path)
scd_p_embd = in_embd_var(scd_path)
thd_p_embd = in_embd_var(thd_path)

# Embed each integer in a 300-dimensional vector as output
rela_embd = layers.Embedding(len(relation2id)+1, 300)(id_rela)

#add 2 layer bi-directional LSTM
lstm_layer_1 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))
lstm_layer_2 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))

#first LSTM layer
fst_lstm_mid = lstm_layer_1(fst_p_embd)
scd_lstm_mid = lstm_layer_1(scd_p_embd)
thd_lstm_mid = lstm_layer_1(thd_p_embd)

#second LSTM layer
fst_lstm_out = lstm_layer_2(fst_lstm_mid)
scd_lstm_out = lstm_layer_2(scd_lstm_mid)
thd_lstm_out = lstm_layer_2(thd_lstm_mid)

#reduce max
fst_reduce_max = tf.reduce_max(fst_lstm_out, axis=1)
scd_reduce_max = tf.reduce_max(scd_lstm_out, axis=1)
thd_reduce_max = tf.reduce_max(thd_lstm_out, axis=1)

#concatenate the output vector from both siamese tunnel: (Batch, 900)
path_concat = layers.concatenate([fst_reduce_max, scd_reduce_max, thd_reduce_max], axis=-1)

#add dropout on top of the concatenation from all channels
dropout = layers.Dropout(0.25)(path_concat)

#multiply into output embd size by dense layer: (Batch, 300)
path_out_vect = layers.Dense(300, activation='tanh')(dropout)

#remove the time dimension from the output embd since there is only one step
rela_out_embd = tf.reduce_sum(rela_embd, axis=1)

# Normalize the vectors to have unit length
path_out_vect_norm = tf.math.l2_normalize(path_out_vect, axis=-1)
rela_out_embd_norm = tf.math.l2_normalize(rela_out_embd, axis=-1)

# Calculate the dot product
dot_product = layers.Dot(axes=-1)([path_out_vect_norm, rela_out_embd_norm])

#put together the model
model = keras.Model([fst_path, scd_path, thd_path, id_rela], dot_product)

2023-05-15 13:37:22.889519: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
#config the Adam optimizer 
opt = keras.optimizers.Adam(learning_rate=0.0005, decay=1e-6)

#compile the model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])

#### Build the subgraph-based siamese neural network

In [13]:
#each input is an vector with number of relations to be dim:
#each dim represent the existence (1) or not (0) of an out-going relation from the entity
source_path_1 = keras.Input(shape=(None,), dtype="int32")
source_path_2 = keras.Input(shape=(None,), dtype="int32")
source_path_3 = keras.Input(shape=(None,), dtype="int32")

target_path_1 = keras.Input(shape=(None,), dtype="int32")
target_path_2 = keras.Input(shape=(None,), dtype="int32")
target_path_3 = keras.Input(shape=(None,), dtype="int32")

#the relation input layer (for output embedding)
id_rela_ = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 300-dimensional vector as input,
# note that we add another "space holder" embedding, 
# which hold the spaces if the initial length of paths are not the same
in_embd_var_ = layers.Embedding(len(relation2id)+1, 300)

# Obtain the source embeddings
source_embd_1 = in_embd_var_(source_path_1)
source_embd_2 = in_embd_var_(source_path_2)
source_embd_3 = in_embd_var_(source_path_3)

#Obtain the target embeddings
target_embd_1 = in_embd_var_(target_path_1)
target_embd_2 = in_embd_var_(target_path_2)
target_embd_3 = in_embd_var_(target_path_3)

# Embed each integer in a 300-dimensional vector as output
rela_embd_ = layers.Embedding(len(relation2id)+1, 300)(id_rela_)

#add 2 layer bi-directional LSTM network
lstm_1 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))
lstm_2 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))

###source lstm implimentation########
#first LSTM layer
source_mid_1 = lstm_1(source_embd_1)
source_mid_2 = lstm_1(source_embd_2)
source_mid_3 = lstm_1(source_embd_3)

#second LSTM layer
source_out_1 = lstm_2(source_mid_1)
source_out_2 = lstm_2(source_mid_2)
source_out_3 = lstm_2(source_mid_3)

#reduce max
source_max_1 = tf.reduce_max(source_out_1, axis=1)
source_max_2 = tf.reduce_max(source_out_2, axis=1)
source_max_3 = tf.reduce_max(source_out_3, axis=1)

#concatenate the output vector from both siamese tunnel: (Batch, 900)
source_concat = layers.concatenate([source_max_1, source_max_2, source_max_3], axis=-1)

#add dropout on top of the concatenation from all channels
source_dropout = layers.Dropout(0.25)(source_concat)

###target lstm implimentation########
#first LSTM layer
target_mid_1 = lstm_1(target_embd_1)
target_mid_2 = lstm_1(target_embd_2)
target_mid_3 = lstm_1(target_embd_3)

#second LSTM layer
target_out_1 = lstm_2(target_mid_1)
target_out_2 = lstm_2(target_mid_2)
target_out_3 = lstm_2(target_mid_3)

#reduce max
target_max_1 = tf.reduce_max(target_out_1, axis=1)
target_max_2 = tf.reduce_max(target_out_2, axis=1)
target_max_3 = tf.reduce_max(target_out_3, axis=1)

#concatenate the output vector from both siamese tunnel: (Batch, 900)
target_concat = layers.concatenate([target_max_1, target_max_2, target_max_3], axis=-1)

#add dropout on top of the concatenation from all channels
target_dropout = layers.Dropout(0.25)(target_concat)

#further concatenate source and target output embeddings: (Batch, 1800)
final_concat = layers.concatenate([source_dropout, target_dropout], axis=-1)

#multiply into output embd size by dense layer: (Batch, 300)
out_vect = layers.Dense(300, activation='tanh')(final_concat)

#remove the time dimension from the output embd since there is only one step
rela_out_embd_ = tf.reduce_sum(rela_embd_, axis=1)

# Normalize the vectors to have unit length
out_vect_norm = tf.math.l2_normalize(out_vect, axis=-1)
rela_out_embd_norm_ = tf.math.l2_normalize(rela_out_embd_, axis=-1)

# Calculate the dot product
dot_product_ = layers.Dot(axes=-1)([out_vect_norm, rela_out_embd_norm_])

#put together the model
model_2 = keras.Model([source_path_1, source_path_2, source_path_3,
                       target_path_1, target_path_2, target_path_3, id_rela_], dot_product_)

In [14]:
#config the Adam optimizer 
opt_ = keras.optimizers.Adam(learning_rate=0.0005, decay=1e-6)

#compile the model
model_2.compile(loss='binary_crossentropy', optimizer=opt_, metrics=['binary_accuracy'])

### Build the big-batch for path-based model
We will build the big-batch for the path-based model training. That is, we will build three list to store three paths, respectively.

In order to reduce computational complexity, we will run the path-finding algorithm for each entity e in the dataset before the training. That is, for each entity e, we will have two dictionaries. Dict 1 stores the paths between e and any other entities in the dataset. Will Dict 2 stores the paths between e and its direct neighbors. The two dicts will be used and invariant throughout the training.

* At each step, three different paths between two entities s and t are selected. Each path is append to one of the list. 
* If this step is for positive samples, the existing relation r will be selected between s and t. If there are more than one relation from s to t, we randomly choose one. Also, the label list will be appended 1.
* If this step is for negative samples, one relation that does not exist between s and t will be selected randomly and append to the relation list. Also, the label list will be appended 0.
* In practice, the positive step is always fallowed by a negative step. The same paths in the positive step will be used in the next negative step, while the relation is a negative one chosen in the above way.
* We do this until the length limit is reached.

**For relation prediciton, we will only need to train using (s,r,t) triple. (t,r-1,s) is not necessary and hence not included in training.**

In [15]:
#function to build the big batche for path-based training
def build_big_batches_path(lower_bd, upper_bd, data, one_hop, s_t_r,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    #the set of all initial relations
    ini_r_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
        
        if i % 2 == 0: #initial relation id is always an even number
            ini_r_id_set.add(i)
    
    num_r = len(id2relation)
    num_ini_r = len(ini_r_id_set)
    
    if num_ini_r != int(num_r/2):
        raise ValueError('error when generating id2relation')
    
    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)
    random.shuffle(existing_ids)
    
    count = 0
    for s in existing_ids:
        
        #impliment the path finding algorithm to find paths between s and t
        result, length_dict = Class_2.obtain_paths('direct_neighbour', s, 'nb', lower_bd, upper_bd, one_hop)
        
        for iteration in range(10):

            #proceed only if at least three paths are between s and t
            for t in result:

                if len(s_t_r[(s,t)]) == 0:

                    raise ValueError(s,t,id2entity[s], id2entity[t])

                #we are only interested in forward link in relation prediciton
                ini_r_list = list()

                #obtain initial relations between s and t
                for r in s_t_r[(s,t)]:
                    if r % 2 == 0:#initial relation id is always an even number
                        ini_r_list.append(r)

                #if there exist more than three paths between s and t, 
                #and inital connection between s and t exists,
                #and not every r in the relation dictionary exists between s and t (although this is rare)
                #we then proceed
                if len(result[t]) >= 3 and len(ini_r_list) > 0 and len(ini_r_list) < int(num_ini_r):

                    #obtain the list form of all the paths from s to t
                    temp_path_list = list(result[t])

                    temp_pair = random.sample(temp_path_list, 3)

                    path_1, path_2, path_3 = temp_pair[0], temp_pair[1], temp_pair[2]

                    #####positive#####################
                    #append the paths: note that we add the space holder id at the end of the shorter path
                    x_p_list['1'].append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
                    x_p_list['2'].append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
                    x_p_list['3'].append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))

                    #append relation
                    r = random.choice(ini_r_list)
                    x_r_list.append([r])
                    y_list.append(1.)

                    #####negative#####################
                    #append the paths: note that we add the space holder id at the end
                    #of the shorter path
                    x_p_list['1'].append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
                    x_p_list['2'].append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
                    x_p_list['3'].append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))

                    #append relation
                    neg_r_list = list(ini_r_id_set.difference(set(ini_r_list)))
                    r_ran = random.choice(neg_r_list)
                    x_r_list.append([r_ran])
                    y_list.append(0.)
        
        count += 1
        if count % 100 == 0:
            print('generating big-batches for path-based model', count, len(existing_ids))

### Build the big-batch for the subgraph-based network training

Again, to reduce computational complexity, we store the subgraph of each entity e at the biginning.

* At each step, we will select one triple (s,r,t) from the dataset. Then, reaching out paths of s and t is generated respectively according to their out-going relations.
* We will select three paths for each of source and target entity. Add them to the corresponding list.
* If this is a positive sample step, the id of relation r is appended to the relation list.
* If this is a negative sample step, the id of a random relation is appended to the relation lsit.
* Similarly, one negative sample step always follows one positive step. The one-hop vectors from the previous positve step is used again for the negative step.

In [16]:
#Again, it is too slow to run the path-finding algorithm again and again on the complete FB15K-237
#Instead, we will find the subgraph for each entity once.
#then in the subgraph based training, the subgraphs are stored and used for multiple times
def store_subgraph_dicts(lower_bd, upper_bd, data, one_hop, s_t_r,
                         relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
    
    num_r = len(id2relation)
    
    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
    
    #the ids to start path finding
    existing_ids = list(existing_ids)
    random.shuffle(existing_ids)
    
    #Dict stores the subgraph for each entity
    Dict_1 = dict()
    
    count = 0
    for s in existing_ids:
        
        path_set = set()
            
        result, length_dict = Class_2.obtain_paths('any_target', s, 'any', lower_bd, upper_bd, one_hop)

        for t_ in result:
            for path in result[t_]:
                path_set.add(path)

        del(result, length_dict)
        
        path_list = list(path_set)
        
        path_select = random.sample(path_list, min(len(path_list), 100))
            
        Dict_1[s] = deepcopy(path_select)
        
        count += 1
        if count % 100 == 0:
            print('generating and storing paths for the path-based model', count, len(existing_ids))
        
    return(Dict_1)

In [17]:
#function to build the big-batch for one-hope neighbor training
def build_big_batches_subgraph(lower_bd, upper_bd, data, one_hop, s_t_r,
                      x_s_list, x_t_list, x_r_list, y_list, Dict,
                      relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    #the set of all initial relations
    ini_r_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
        
        if i % 2 == 0: #initial relation id is always an even number
            ini_r_id_set.add(i)
    
    num_r = len(id2relation)
    num_ini_r = len(ini_r_id_set)
    
    if num_ini_r != int(num_r/2):
        raise ValueError('error when generating id2relation')
        
    #if an entity has at least three out-stretching paths, it is a qualified one
    qualified = set()
    for e in Dict:
        if len(Dict[e]) >= 3:
            qualified.add(e)
    qualified = list(qualified)
    
    data = list(data)
    
    for iteration in range(10):

        data = shuffle(data)

        for i_0 in range(len(data)):

            triple = data[i_0]

            s, r, t = triple[0], triple[1], triple[2] #obtain entities and relation IDs

            if s in qualified and t in qualified:

                #obtain the path list for true entities
                path_s, path_t = list(Dict[s]), list(Dict[t])

                #####positive step###########
                #randomly obtain three paths for true entities
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative step for relation###########
                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                neg_r_list = list(ini_r_id_set.difference({r}))
                r_ran = random.choice(neg_r_list)
                x_r_list.append([r_ran])
                y_list.append(0.)
                
                ##############################################
                ##############################################
                #randomly choose two negative sampled entities
                s_ran = random.choice(qualified)
                t_ran = random.choice(qualified)

                #obtain the path list for random entities
                path_s_ran, path_t_ran = list(Dict[s_ran]), list(Dict[t_ran])
                
                #####positive step#################
                #Again: randomly obtain three paths
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative for source entity###########
                #randomly obtain three paths
                temp_s = random.sample(path_s_ran, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(0.)

                #####positive step###########
                #Again: randomly obtain three paths
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative for target entity###########
                #randomly obtain three paths
                temp_t = random.sample(path_t_ran, 3)
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(0.)

            if i_0 % 200 == 0:
                print('generating big-batches for subgraph-based model', i_0, len(data), iteration)

### Start Training: load the KG and call classes

Here, we use the validation set to see the training efficiency. That is, we use the validation to check whether the true relation between entities can be predicted by paths.

The trick is: in validation, we have to use the same relation ID and entity ID as in the training. But we don't want to use the links in training anymore. That is, in validation, we want to use (and update if necessary) entity2id, id2entity, relation2id and id2relation. But we want to use new one_hop, data, data_ and s_t_r for validation set. Then, path-finding will also be based on new one_hop.


In [18]:
model_name

'Model_SiaLP_3_new_WN18RR_v4'

In [19]:
one_hop_model_name

'One_hop_model_SiaLP_3_new_WN18RR_v4'

In [20]:
ids_name

'IDs_SiaLP_3_new_WN18RR_v4'

In [21]:
#first, we save the relation and ids
Dict = dict()

#save training data
Dict['one_hop'] = one_hop
Dict['data'] = data
Dict['s_t_r'] = s_t_r

#save valid data
Dict['one_hop_valid'] = one_hop_valid
Dict['data_valid'] = data_valid
Dict['s_t_r_valid'] = s_t_r_valid

#save test data
Dict['one_hop_test'] = one_hop_test
Dict['data_test'] = data_test
Dict['s_t_r_test'] = s_t_r_test

#save shared dictionaries
Dict['entity2id'] = entity2id
Dict['id2entity'] = id2entity
Dict['relation2id'] = relation2id
Dict['id2relation'] = id2relation

with open('../weight_bin/' + ids_name + '.pickle', 'wb') as handle:
    pickle.dump(Dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
###train the path-based model
lower_bd = lower_bound
upper_bd = upper_bound_path
num_epoch = 10
batch_size = 32
        
#define the training lists
train_p_list, train_r_list, train_y_list = {'1': [], '2': [], '3': []}, list(), list()

#define the validation lists
valid_p_list, valid_r_list, valid_y_list = {'1': [], '2': [], '3': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches_path(lower_bd, upper_bd, data, one_hop, s_t_r,
                      train_p_list, train_r_list, train_y_list,
                      relation2id, entity2id, id2relation, id2entity)

#fill in the validation array list
build_big_batches_path(lower_bd, upper_bd, data_valid, one_hop_valid, s_t_r_valid,
                      valid_p_list, valid_r_list, valid_y_list,
                      relation2id, entity2id, id2relation, id2entity)    

#######################################
###do the training#####################
#sometimes the validation dataset is so small so sparse, 
#which cannot find three paths between any pair of s and t.
#in such a case, we will divide the training big-batch into train and valid
if len(valid_y_list) >= 100:
    #generate the input arrays
    x_train_1 = np.asarray(train_p_list['1'], dtype='int')
    x_train_2 = np.asarray(train_p_list['2'], dtype='int')
    x_train_3 = np.asarray(train_p_list['3'], dtype='int')
    x_train_r = np.asarray(train_r_list, dtype='int')
    y_train = np.asarray(train_y_list, dtype='int')

    #generate the validation arrays
    x_valid_1 = np.asarray(valid_p_list['1'], dtype='int')
    x_valid_2 = np.asarray(valid_p_list['2'], dtype='int')
    x_valid_3 = np.asarray(valid_p_list['3'], dtype='int')
    x_valid_r = np.asarray(valid_r_list, dtype='int')
    y_valid = np.asarray(valid_y_list, dtype='int')

else:
    split = int(len(train_y_list)*0.8)
    #generate the input arrays
    x_train_1 = np.asarray(train_p_list['1'][:split], dtype='int')
    x_train_2 = np.asarray(train_p_list['2'][:split], dtype='int')
    x_train_3 = np.asarray(train_p_list['3'][:split], dtype='int')
    x_train_r = np.asarray(train_r_list[:split], dtype='int')
    y_train = np.asarray(train_y_list[:split], dtype='int')

    #generate the validation arrays
    x_valid_1 = np.asarray(train_p_list['1'][split:], dtype='int')
    x_valid_2 = np.asarray(train_p_list['2'][split:], dtype='int')
    x_valid_3 = np.asarray(train_p_list['3'][split:], dtype='int')
    x_valid_r = np.asarray(train_r_list[split:], dtype='int')
    y_valid = np.asarray(train_y_list[split:], dtype='int')

#do the training
model.fit([x_train_1, x_train_2, x_train_3, x_train_r], y_train, 
          validation_data=([x_valid_1, x_valid_2, x_valid_3, x_valid_r], y_valid),
          batch_size=batch_size, epochs=num_epoch)   

# Save model and weights
add_h5 = model_name + '.h5'
save_dir = os.path.join(os.getcwd(), '../weight_bin')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, add_h5)
model.save(model_path)
print('Save model')
del(model)

generating big-batches for path-based model 100 3861
generating big-batches for path-based model 200 3861
generating big-batches for path-based model 300 3861
generating big-batches for path-based model 400 3861
generating big-batches for path-based model 500 3861
generating big-batches for path-based model 600 3861
generating big-batches for path-based model 700 3861
generating big-batches for path-based model 800 3861
generating big-batches for path-based model 900 3861
generating big-batches for path-based model 1000 3861
generating big-batches for path-based model 1100 3861
generating big-batches for path-based model 1200 3861
generating big-batches for path-based model 1300 3861
generating big-batches for path-based model 1400 3861
generating big-batches for path-based model 1500 3861
generating big-batches for path-based model 1600 3861
generating big-batches for path-based model 1700 3861
generating big-batches for path-based model 1800 3861
generating big-batches for path-based

In [23]:
###train the subgraph-based model
lower_bd = lower_bound
upper_bd = upper_bound_subg
num_epoch = 10
batch_size = 32

Dict_train = store_subgraph_dicts(lower_bd, upper_bd, data, one_hop, s_t_r,
                         relation2id, entity2id, id2relation, id2entity)

Dict_valid = store_subgraph_dicts(lower_bd, upper_bd, data_valid, one_hop_valid, s_t_r_valid,
                         relation2id, entity2id, id2relation, id2entity)
        
#define the training lists
train_s_list, train_t_list, train_r_list, train_y_list = {'1': [], '2': [], '3': []}, {'1': [], '2': [], '3': []}, list(), list()

#define the validation lists
valid_s_list, valid_t_list, valid_r_list, valid_y_list = {'1': [], '2': [], '3': []}, {'1': [], '2': [], '3': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches_subgraph(lower_bd, upper_bd, data, one_hop, s_t_r,
                      train_s_list, train_t_list, train_r_list, train_y_list, Dict_train,
                      relation2id, entity2id, id2relation, id2entity)

#fill in the validation array list
build_big_batches_subgraph(lower_bd, upper_bd, data_valid, one_hop_valid, s_t_r_valid,
                      valid_s_list, valid_t_list, valid_r_list, valid_y_list, Dict_valid,
                      relation2id, entity2id, id2relation, id2entity)    

#######################################
###do the training#####################
#sometimes the validation dataset is so small so sparse, 
#which cannot find three paths between any pair of s and t.
#in such a case, we will divide the training big-batch into train and valid
if len(valid_y_list) >= 100:
    #generate the input arrays
    x_train_s_1 = np.asarray(train_s_list['1'], dtype='int')
    x_train_s_2 = np.asarray(train_s_list['2'], dtype='int')
    x_train_s_3 = np.asarray(train_s_list['3'], dtype='int')

    x_train_t_1 = np.asarray(train_t_list['1'], dtype='int')
    x_train_t_2 = np.asarray(train_t_list['2'], dtype='int')
    x_train_t_3 = np.asarray(train_t_list['3'], dtype='int')

    x_train_r = np.asarray(train_r_list, dtype='int')
    y_train = np.asarray(train_y_list, dtype='int')

    #generate the validation arrays
    x_valid_s_1 = np.asarray(valid_s_list['1'], dtype='int')
    x_valid_s_2 = np.asarray(valid_s_list['2'], dtype='int')
    x_valid_s_3 = np.asarray(valid_s_list['3'], dtype='int')

    x_valid_t_1 = np.asarray(valid_t_list['1'], dtype='int')
    x_valid_t_2 = np.asarray(valid_t_list['2'], dtype='int')
    x_valid_t_3 = np.asarray(valid_t_list['3'], dtype='int')

    x_valid_r = np.asarray(valid_r_list, dtype='int')
    y_valid = np.asarray(valid_y_list, dtype='int')

else:
    split = int(len(train_y_list)*0.8)
    #generate the input arrays
    x_train_s_1 = np.asarray(train_s_list['1'][:split], dtype='int')
    x_train_s_2 = np.asarray(train_s_list['2'][:split], dtype='int')
    x_train_s_3 = np.asarray(train_s_list['3'][:split], dtype='int')

    x_train_t_1 = np.asarray(train_t_list['1'][:split], dtype='int')
    x_train_t_2 = np.asarray(train_t_list['2'][:split], dtype='int')
    x_train_t_3 = np.asarray(train_t_list['3'][:split], dtype='int')

    x_train_r = np.asarray(train_r_list[:split], dtype='int')
    y_train = np.asarray(train_y_list[:split], dtype='int')

    #generate the validation arrays
    x_valid_s_1 = np.asarray(train_s_list['1'][split:], dtype='int')
    x_valid_s_2 = np.asarray(train_s_list['2'][split:], dtype='int')
    x_valid_s_3 = np.asarray(train_s_list['3'][split:], dtype='int')

    x_valid_t_1 = np.asarray(train_t_list['1'][split:], dtype='int')
    x_valid_t_2 = np.asarray(train_t_list['2'][split:], dtype='int')
    x_valid_t_3 = np.asarray(train_t_list['3'][split:], dtype='int')

    x_valid_r = np.asarray(train_r_list[split:], dtype='int')
    y_valid = np.asarray(train_y_list[split:], dtype='int')

#do the training
model_2.fit([x_train_s_1, x_train_s_2, x_train_s_3, x_train_t_1, x_train_t_2, x_train_t_3, x_train_r], y_train, 
          validation_data=([x_valid_s_1, x_valid_s_2, x_valid_s_3, x_valid_t_1, x_valid_t_2, x_valid_t_3, x_valid_r], y_valid),
          batch_size=batch_size, epochs=num_epoch)

# Save model and weights
one_hop_add_h5 = one_hop_model_name + '.h5'
one_hop_save_dir = os.path.join(os.getcwd(), '../weight_bin')

if not os.path.isdir(one_hop_save_dir):
    os.makedirs(one_hop_save_dir)
one_hop_model_path = os.path.join(one_hop_save_dir, one_hop_add_h5)
model_2.save(one_hop_model_path)
print('Save model')
del(model_2, Dict_train, Dict_valid)

generating and storing paths for the path-based model 100 3861
generating and storing paths for the path-based model 200 3861
generating and storing paths for the path-based model 300 3861
generating and storing paths for the path-based model 400 3861
generating and storing paths for the path-based model 500 3861
generating and storing paths for the path-based model 600 3861
generating and storing paths for the path-based model 700 3861
generating and storing paths for the path-based model 800 3861
generating and storing paths for the path-based model 900 3861
generating and storing paths for the path-based model 1000 3861
generating and storing paths for the path-based model 1100 3861
generating and storing paths for the path-based model 1200 3861
generating and storing paths for the path-based model 1300 3861
generating and storing paths for the path-based model 1400 3861
generating and storing paths for the path-based model 1500 3861
generating and storing paths for the path-based m

generating big-batches for subgraph-based model 1400 7940 2
generating big-batches for subgraph-based model 1600 7940 2
generating big-batches for subgraph-based model 1800 7940 2
generating big-batches for subgraph-based model 2000 7940 2
generating big-batches for subgraph-based model 2200 7940 2
generating big-batches for subgraph-based model 2400 7940 2
generating big-batches for subgraph-based model 2600 7940 2
generating big-batches for subgraph-based model 2800 7940 2
generating big-batches for subgraph-based model 3000 7940 2
generating big-batches for subgraph-based model 3200 7940 2
generating big-batches for subgraph-based model 3400 7940 2
generating big-batches for subgraph-based model 3600 7940 2
generating big-batches for subgraph-based model 3800 7940 2
generating big-batches for subgraph-based model 4000 7940 2
generating big-batches for subgraph-based model 4200 7940 2
generating big-batches for subgraph-based model 4400 7940 2
generating big-batches for subgraph-base

generating big-batches for subgraph-based model 200 7940 6
generating big-batches for subgraph-based model 400 7940 6
generating big-batches for subgraph-based model 600 7940 6
generating big-batches for subgraph-based model 800 7940 6
generating big-batches for subgraph-based model 1000 7940 6
generating big-batches for subgraph-based model 1200 7940 6
generating big-batches for subgraph-based model 1400 7940 6
generating big-batches for subgraph-based model 1600 7940 6
generating big-batches for subgraph-based model 1800 7940 6
generating big-batches for subgraph-based model 2000 7940 6
generating big-batches for subgraph-based model 2200 7940 6
generating big-batches for subgraph-based model 2400 7940 6
generating big-batches for subgraph-based model 2600 7940 6
generating big-batches for subgraph-based model 2800 7940 6
generating big-batches for subgraph-based model 3000 7940 6
generating big-batches for subgraph-based model 3200 7940 6
generating big-batches for subgraph-based mo

generating big-batches for subgraph-based model 4000 7940 9
generating big-batches for subgraph-based model 4200 7940 9
generating big-batches for subgraph-based model 4400 7940 9
generating big-batches for subgraph-based model 4600 7940 9
generating big-batches for subgraph-based model 4800 7940 9
generating big-batches for subgraph-based model 5000 7940 9
generating big-batches for subgraph-based model 5200 7940 9
generating big-batches for subgraph-based model 5400 7940 9
generating big-batches for subgraph-based model 5600 7940 9
generating big-batches for subgraph-based model 5800 7940 9
generating big-batches for subgraph-based model 6000 7940 9
generating big-batches for subgraph-based model 6200 7940 9
generating big-batches for subgraph-based model 6400 7940 9
generating big-batches for subgraph-based model 6600 7940 9
generating big-batches for subgraph-based model 6800 7940 9
generating big-batches for subgraph-based model 7000 7940 9
generating big-batches for subgraph-base

### Result on the testset for inductive link prediction

We use the testset for inductive link prediction.

In [1]:
data_name = 'WN18RR_v4'
model_id = 'SiaLP_3_new'
lower_bound = 1
upper_bound_path = 10
upper_bound_subg = 3

In [2]:
#difine the names for saving
model_name = 'Model_' + model_id + '_' + data_name
one_hop_model_name = 'One_hop_model_' + model_id + '_' + data_name
ids_name = 'IDs_' + model_id + '_' + data_name

In [3]:
ids_name

'IDs_SiaLP_3_new_WN18RR_v4'

In [4]:
one_hop_model_name

'One_hop_model_SiaLP_3_new_WN18RR_v4'

In [5]:
model_name

'Model_SiaLP_3_new_WN18RR_v4'

In [6]:
import librosa
import opensmile
import os
import sys
import numpy as np
import random
import pickle

from collections import defaultdict
from copy import deepcopy
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import initializers
from tensorflow.keras.utils import plot_model

In [7]:
class LoadKG:
    
    def __init__(self):
        
        self.x = 'Hello'
        
    def load_train_data(self, data_path, one_hop, data, s_t_r, entity2id, id2entity,
                     relation2id, id2relation):
        
        data_ = set()
    
        ####load the train, valid and test set##########
        with open (data_path, 'r') as f:
            
            data_ini = f.readlines()
                        
            for i in range(len(data_ini)):
            
                x = data_ini[i].split()
                
                x_ = tuple(x)
                
                data_.add(x_)
        
        ####relation dict#################
        index = len(relation2id)
     
        for key in data_:
            
            if key[1] not in relation2id:
                
                relation = key[1]
                
                relation2id[relation] = index
                
                id2relation[index] = relation
                
                index += 1
                
                #the inverse relation
                iv_r = '_inverse_' + relation
                
                relation2id[iv_r] = index
                
                id2relation[index] = iv_r
                
                index += 1
        
        #get the id of the inverse relation, by above definition, initial relation has 
        #always even id, while inverse relation has always odd id.
        def inverse_r(r):
            
            if r % 2 == 0: #initial relation
                
                iv_r = r + 1
            
            else: #inverse relation
                
                iv_r = r - 1
            
            return(iv_r)
        
        ####entity dict###################
        index = len(entity2id)
        
        for key in data_:
            
            source, target = key[0], key[2]
            
            if source not in entity2id:
                                
                entity2id[source] = index
                
                id2entity[index] = source
                
                index += 1
            
            if target not in entity2id:
                
                entity2id[target] = index
                
                id2entity[index] = target
                
                index += 1
                
        #create the set of triples using id instead of string        
        for ele in data_:
            
            s = entity2id[ele[0]]
            
            r = relation2id[ele[1]]
            
            t = entity2id[ele[2]]
            
            if (s,r,t) not in data:
                
                data.add((s,r,t))
            
            s_t_r[(s,t)].add(r)
            
            if s not in one_hop:
                
                one_hop[s] = set()
            
            one_hop[s].add((r,t))
            
            if t not in one_hop:
                
                one_hop[t] = set()
            
            r_inv = inverse_r(r)
            
            s_t_r[(t,s)].add(r_inv)
            
            one_hop[t].add((r_inv,s))
            
        #change each set in one_hop to list
        for e in one_hop:
            
            one_hop[e] = list(one_hop[e])

In [8]:
class ObtainPathsByDynamicProgramming:

    def __init__(self, amount_bd=50, size_bd=50, threshold=20000):
        
        self.amount_bd = amount_bd #how many Tuples we choose in one_hop[node] for next recursion
                        
        self.size_bd = size_bd #size bound limit the number of paths to a target entity t
        
        #number of times paths with specific length been performed for recursion
        self.threshold = threshold
        
    '''
    Given an entity s, the function will find the paths from s to other entities, using recursion.
    
    One may refer to LeetCode Problem 797 for details:
        https://leetcode.com/problems/all-paths-from-source-to-target/
    '''
    def obtain_paths(self, mode, s, t_input, lower_bd, upper_bd, one_hop):

        if type(lower_bd) != type(1) or lower_bd < 1:
            
            raise TypeError("!!! invalid lower bound setting, must >= 1 !!!")
            
        if type(upper_bd) != type(1) or upper_bd < 1:
            
            raise TypeError("!!! invalid upper bound setting, must >= 1 !!!")
            
        if lower_bd > upper_bd:
            
            raise TypeError("!!! lower bound must not exced upper bound !!!")
            
        if s not in one_hop:
            
            raise ValueError('!!! entity not in one_hop. Please work on existing entities')

        #here is the result dict. Its key is each entity t sharing paths from s
        #The value of each t is a set containing the paths from s to t
        #These paths can be either the direct connection r, or a multi-hop path
        res = defaultdict(set)
        
        #qualified_t contains the types of t we want to consider,
        #that is, what t will be added to the result set.
        qualified_t = set()

        #under this mode, we will only consider the direct neighbour of s
        if mode == 'direct_neighbour':
        
            for Tuple in one_hop[s]:
            
                t = Tuple[1]
                
                qualified_t.add(t)
        
        #under this mode, we will only consider one specified entity t
        elif mode == 'target_specified':
            
            qualified_t.add(t_input)
        
        #under this mode, we will consider any entity
        elif mode == 'any_target':
            
            for s_any in one_hop:
                
                qualified_t.add(s_any)
                
        else:
            
            raise ValueError('not a valid mode')
        
        '''
        We use recursion to find the paths
        On current node with the path [r1, ..., rk] and on-path entities {s, e1, ..., ek-1, node}
        from s to this node, we will further find the direct neighbor t' of this node. 
        If t' is not an on-path entity (not among s, e1,...ek-1, node), we recursively proceed to t' 
        '''
        def helper(node, path, on_path_en, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict):

            #when the current path is within lower_bd and upper_bd, 
            #and the node is among the qualified t, and it has not been fill of paths w.r.t size_limit,
            #we will add this path to the node
            if (len(path) >= lower_bd) and (len(path) <= upper_bd) and (
                node in qualified_t) and (len(res[node]) < self.size_bd):
                
                res[node].add(tuple(path))
                    
            #won't start new recursions if the current path length already reaches upper limit
            #or the number of recursions performed on this length has reached the limit
            if (len(path) < upper_bd) and (count_dict[len(path)] <= self.threshold):
                                
                #temp list is the id list for us to go-over one_hop[node]
                temp_list = [i for i in range(len(one_hop[node]))]
                random.shuffle(temp_list) #so we random-shuffle the list
                
                #only take 20 recursions if there are too many (r,t)
                for i in temp_list[:self.amount_bd]:
                    
                    #obtain tuple of (r,t)
                    Tuple = one_hop[node][i]
                    r, t = Tuple[0], Tuple[1]
                    
                    #add to count_dict even if eventually this step not proceed
                    count_dict[len(path)] += 1
                    
                    #if t not on the path and we not exceed the computation threshold, 
                    #then finally proceed to next recursion
                    if (t not in on_path_en) and (count_dict[len(path)] <= self.threshold):

                        helper(t, path + [r], on_path_en.union({t}), res, qualified_t, 
                               lower_bd, upper_bd, one_hop, count_dict)

        length_dict = defaultdict(int)
        count_dict = defaultdict(int)
        
        helper(s, [], {s}, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict)
        
        return(res, count_dict)

In [9]:
#load the classes
Class_1 = LoadKG()
Class_2 = ObtainPathsByDynamicProgramming()

In [10]:
#load ids and relation/entity dicts
with open('../weight_bin/' + ids_name + '.pickle', 'rb') as handle:
    Dict = pickle.load(handle)
    
#save training data
one_hop = Dict['one_hop']
data = Dict['data']
s_t_r = Dict['s_t_r']

#save valid data
one_hop_valid = Dict['one_hop_valid']
data_valid = Dict['data_valid']
s_t_r_valid = Dict['s_t_r_valid']

#save test data
one_hop_test = Dict['one_hop_test']
data_test = Dict['data_test']
s_t_r_test = Dict['s_t_r_test']

#save shared dictionaries
entity2id = Dict['entity2id']
id2entity = Dict['id2entity']
relation2id = Dict['relation2id']
id2relation = Dict['id2relation']

#we want to keep the initial entity/relation dicts before adding new entities
entity2id_ini = deepcopy(entity2id)
id2entity_ini = deepcopy(id2entity)
relation2id_ini = deepcopy(relation2id)
id2relation_ini = deepcopy(id2relation)

num_r = len(id2relation)
num_r

18

In [11]:
ids_name

'IDs_SiaLP_3_new_WN18RR_v4'

In [12]:
model_name

'Model_SiaLP_3_new_WN18RR_v4'

In [13]:
#load the model
model = keras.models.load_model('../weight_bin/' + model_name + '.h5')

2023-05-15 19:20:35.202344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
#load the one-hop neighbor model
model_2 = keras.models.load_model('../weight_bin/' + one_hop_model_name + '.h5')

In [15]:
ind_train_path = '../data/' + data_name + '_ind/train.txt'
ind_valid_path = '../data/' + data_name + '_ind/valid.txt'
ind_test_path = '../data/' + data_name + '_ind/test.txt'

In [16]:
#load the test dataset
one_hop_ind = dict() 
data_ind = set()
s_t_r_ind = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_train_path, 
                        one_hop_ind, data_ind, s_t_r_ind,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [17]:
print(size_0, size_1, len(data_ind))

3861 10945 12334


In [18]:
#load the test dataset
one_hop_ind_test = dict() 
data_ind_test = set()
s_t_r_ind_test = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_test_path, 
                        one_hop_ind_test, data_ind_test, s_t_r_ind_test,
                        entity2id, id2entity, relation2id, id2relation)


len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [19]:
print(size_0, size_1, len(data_ind_test))

10945 10945 1429


In [20]:
#load the validation for existing triple removal when ranking
one_hop_ind_valid = dict() 
data_ind_valid = set()
s_t_r_ind_valid = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_valid_path, 
                        one_hop_ind_valid, data_ind_valid, s_t_r_ind_valid,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [21]:
print(size_0, size_1, len(data_ind_valid))

10945 10945 1394


In [22]:
print(len(entity2id), len(entity2id_ini))

10945 3861


In [23]:
#obtain all the inital entities and new entities
ini_ent_set, new_ent_set, all_ent_set = set(), set(), set()

for ID in id2entity:
    all_ent_set.add(ID)
    if ID in id2entity_ini:
        ini_ent_set.add(ID)
    else:
        new_ent_set.add(ID)
        
print(len(ini_ent_set), len(new_ent_set), len(all_ent_set))

3861 7084 10945


In [24]:
#we want to check whether there are overlapping 
#between the entities of train triples and inductive test and valid triples
overlapping = 0

for ele in data_ind_test:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

In [25]:
overlapping = 0

for ele in data_ind_valid:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

In [26]:
#we want to check whether there are overlapping 
#between the entities of train triples and inductive test and valid triples
overlapping = 0

for ele in data_ind:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

In [27]:
#the function to do path-based relation scoring
def path_based_relation_scoring(s, t, lower_bd, upper_bd, one_hop, id2relation, model):
    
    path_holder = set()
    
    for iteration in range(3):
    
        result, length_dict = Class_2.obtain_paths('target_specified', 
                                                   s, t, lower_bd, upper_bd, one_hop)
        if t in result:
            
            for path in result[t]:
                
                path_holder.add(path)
                
        del(result, length_dict)
    
    path_holder = list(path_holder)
    random.shuffle(path_holder)
    
    score_dict = defaultdict(float)
    count_dict = defaultdict(int)
    
    count = 0
    
    if len(path_holder) >= 3:
    
        #iterate over path_1
        while count < 10:

            temp_pair = random.sample(path_holder, 3)

            path_1, path_2, path_3 = temp_pair[0], temp_pair[1], temp_pair[2]

            list_1 = list()
            list_2 = list()
            list_3 = list()
            list_r = list()

            for i in range(len(id2relation)):

                if i not in id2relation:

                    raise ValueError ('error when generating id2relation')
                
                #only care about initial relations
                if i % 2 == 0:

                    list_1.append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
                    list_2.append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
                    list_3.append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))
                    list_r.append([i])
            
            #change to arrays
            input_1 = np.array(list_1)
            input_2 = np.array(list_2)
            input_3 = np.array(list_3)
            input_r = np.array(list_r)

            pred = model.predict([input_1, input_2, input_3, input_r], verbose = 0)

            for i in range(pred.shape[0]):
                #need to times 2 to go back to relation id from pred position
                score_dict[2*i] += float(pred[i])
                count_dict[2*i] += 1

            count += 1
            
    #average the score
    for r in score_dict:
        score_dict[r] = deepcopy(score_dict[r]/float(count_dict[r]))
    
    print(len(score_dict), len(path_holder))

    return(score_dict)

In [28]:
#the function to do path-based triple scoring: input one triple
def path_based_triple_scoring(s, r, t, lower_bd, upper_bd, one_hop, id2relation, model):
    
    path_holder = set()
    
    for iteration in range(3):
    
        result, length_dict = Class_2.obtain_paths('target_specified', 
                                                   s, t, lower_bd, upper_bd, one_hop)
        if t in result:
            
            for path in result[t]:
                
                path_holder.add(path)
                
        del(result, length_dict)
    
    path_holder = list(path_holder)
    random.shuffle(path_holder)
    
    score = 0.
    count = 0
    
    if len(path_holder) >= 3:
        
        list_1 = list()
        list_2 = list()
        list_3 = list()
        list_r = list()
    
        #iterate over path_1
        while count < 10:

            temp_pair = random.sample(path_holder, 3)
            path_1, path_2, path_3 = temp_pair[0], temp_pair[1], temp_pair[2]

            list_1.append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
            list_2.append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
            list_3.append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))
            list_r.append([r])
            
            count += 1
            
        #change to arrays
        input_1 = np.array(list_1)
        input_2 = np.array(list_2)
        input_3 = np.array(list_3)
        input_r = np.array(list_r)

        pred = model.predict([input_1, input_2, input_3, input_r], verbose = 0)

        for i in range(pred.shape[0]):
            score += float(pred[i])
            
        #average the score
        score = score/float(count)

    return(score)

In [29]:
#subgraph based relation scoring
def subgraph_relation_scoring(s, t, lower_bd, upper_bd, one_hop, id2relation, model_2):
    
    path_s, path_t = set(), set() #sets holding all the paths from s or t
    
    for iteration in range(3):
    
        #obtain the paths out from s or t by "any target" mode. That is, 
        result_s, length_dict_s = Class_2.obtain_paths('any_target', s, 'any', lower_bd, upper_bd, one_hop)
        result_t, length_dict_t = Class_2.obtain_paths('any_target', t, 'any', lower_bd, upper_bd, one_hop)

        #add paths to the source/target path_set
        for e in result_s:
            for path in result_s[e]:
                path_s.add(path)
        for e in result_t:
            for path in result_t[e]:
                path_t.add(path)
                
        del(result_s, length_dict_s, result_t, length_dict_t)
    
    #final output: the score dict
    score_dict = defaultdict(float)
    count_dict = defaultdict(int)
    
    #see if both path_s and path_t have at least three paths
    if len(path_s) >= 3 and len(path_t) >= 3:

        #change to lists
        path_s, path_t = list(path_s), list(path_t)
        
        count = 0
        while count < 10:
            
            #lists holding the input to the network
            list_s_1 = list()
            list_s_2 = list()
            list_s_3 = list()
            list_t_1 = list()
            list_t_2 = list()
            list_t_3 = list()
            list_r = list()

            #randomly obtain three paths
            temp_s = random.sample(path_s, 3)
            temp_t = random.sample(path_t, 3)
            s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
            t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]
            
            #add all forward (initial relation)
            for i in range(len(id2relation)):

                if i not in id2relation:

                    raise ValueError ('error when generating id2relation')
                    
                if i % 2 == 0:

                    #append the paths: note that we add the space holder id at the end of the shorter path
                    list_s_1.append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                    list_s_2.append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                    list_s_3.append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))
                    
                    list_t_1.append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                    list_t_2.append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                    list_t_3.append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))
                    
                    list_r.append([i])
                
            #change to arrays
            input_s_1 = np.array(list_s_1)
            input_s_2 = np.array(list_s_2)
            input_s_3 = np.array(list_s_3)
            input_t_1 = np.array(list_t_1)
            input_t_2 = np.array(list_t_2)
            input_t_3 = np.array(list_t_3)
            input_r = np.array(list_r)
            
            pred = model_2.predict([input_s_1, input_s_2, input_s_3,
                                    input_t_1, input_t_2, input_t_3, input_r], verbose = 0)

            for i in range(pred.shape[0]):
                #need to times 2 to go back to relation id from pred position
                score_dict[2*i] += float(pred[i])
                count_dict[2*i] += 1

            count += 1
            
    #average the score
    for r in score_dict:
        score_dict[r] = deepcopy(score_dict[r]/float(count_dict[r]))
            
    print(len(score_dict), len(path_s), len(path_t))
        
    return(score_dict)

In [30]:
#subgraph based triple scoring
def subgraph_triple_scoring(s, r, t, lower_bd, upper_bd, one_hop, id2relation, model_2):
    
    path_s, path_t = set(), set() #sets holding all the paths from s or t
    
    for iteration in range(3):
    
        #obtain the paths out from s or t by "any target" mode. That is, 
        result_s, length_dict_s = Class_2.obtain_paths('any_target', s, 'any', lower_bd, upper_bd, one_hop)
        result_t, length_dict_t = Class_2.obtain_paths('any_target', t, 'any', lower_bd, upper_bd, one_hop)

        #add paths to the source/target path_set
        for e in result_s:
            for path in result_s[e]:
                path_s.add(path)
        for e in result_t:
            for path in result_t[e]:
                path_t.add(path)
                
        del(result_s, length_dict_s, result_t, length_dict_t)
    
    #final output: the score dict
    score = 0.
    
    #see if both path_s and path_t have at least three paths
    if len(path_s) >= 3 and len(path_t) >= 3:

        #change to lists
        path_s, path_t = list(path_s), list(path_t)
        
        #lists holding the input to the network
        list_s_1 = list()
        list_s_2 = list()
        list_s_3 = list()
        list_t_1 = list()
        list_t_2 = list()
        list_t_3 = list()
        list_r = list()
        
        count = 0
        while count < 10:

            #randomly obtain three paths
            temp_s = random.sample(path_s, 3)
            temp_t = random.sample(path_t, 3)
            s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
            t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

            #append the paths: note that we add the space holder id at the end of the shorter path
            list_s_1.append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
            list_s_2.append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
            list_s_3.append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

            list_t_1.append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
            list_t_2.append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
            list_t_3.append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

            list_r.append([r])
            count += 1
                
        #change to arrays
        input_s_1 = np.array(list_s_1)
        input_s_2 = np.array(list_s_2)
        input_s_3 = np.array(list_s_3)
        input_t_1 = np.array(list_t_1)
        input_t_2 = np.array(list_t_2)
        input_t_3 = np.array(list_t_3)
        input_r = np.array(list_r)

        pred = model_2.predict([input_s_1, input_s_2, input_s_3,
                                input_t_1, input_t_2, input_t_3, input_r], verbose = 0)

        for i in range(pred.shape[0]):
            score += float(pred[i])

        #average the score
        score = score/float(count)
        
    return(score)

#### Not fine tuned 

In [31]:
########################################################
#obtain the Hits@N for relation prediction##############

#we select all the triples in the inductive test set
selected = list(data_ind_test)

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0
Hits_at_3 = 0
Hits_at_10 = 0
MRR_raw = 0.

for i in range(len(selected)):
    
    s_true, r_true, t_true = selected[i][0], selected[i][1], selected[i][2]
    
    #run the path-based scoring
    score_dict_path = path_based_relation_scoring(s_true, t_true, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)
    
    #run the one-hop neighbour based scoring
    score_dict_subg = subgraph_relation_scoring(s_true, t_true, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #final score dict
    score_dict = defaultdict(float)
    
    for r in score_dict_path:
        score_dict[r] += score_dict_path[r]
    for r in score_dict_subg:
        score_dict[r] += score_dict_subg[r]
    
    #[... [score, r], ...]
    temp_list = list()
    
    for r in id2relation:
        
        #again, we only care about initial relation prediciton
        if r % 2 == 0:
        
            if r in score_dict:

                temp_list.append([score_dict[r], r])

            else:

                temp_list.append([0.0, r])
        
    sorted_list = sorted(temp_list, key = lambda x: x[0], reverse=True)
    
    p = 0
    exist_tri = 0
    
    while p < len(sorted_list) and sorted_list[p][1] != r_true:
        
        #moreover, we want to remove existing triples
        if ((s_true, sorted_list[p][1], t_true) in data_test) or (
            (s_true, sorted_list[p][1], t_true) in data_valid) or (
            (s_true, sorted_list[p][1], t_true) in data) or (
            (s_true, sorted_list[p][1], t_true) in data_ind) or (
            (s_true, sorted_list[p][1], t_true) in data_ind_valid) or (
            (s_true, sorted_list[p][1], t_true) in data_ind_test):
            
            exist_tri += 1
            
        p += 1
    
    if p - exist_tri == 0:
        
        Hits_at_1 += 1
        
    if p - exist_tri < 3:
        
        Hits_at_3 += 1
        
    if p - exist_tri < 10:
        
        Hits_at_10 += 1
        
    MRR_raw += 1./float(p - exist_tri + 1.) 
        
    print('checkcorrect', r_true, sorted_list[p][1],
          'real score', sorted_list[p][0],
          'Hits@1', Hits_at_1/(i+1),
          'Hits@3', Hits_at_3/(i+1),
          'Hits@10', Hits_at_10/(i+1),
          'MRR', MRR_raw/(i+1),
          'cur_rank', p - exist_tri,
          'abs_cur_rank', p,
          'total_num', i, len(selected))

9 82
9 27 51
checkcorrect 2 2 real score 1.7105379343032836 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 0 1429
9 132
9 20 26
checkcorrect 2 2 real score 1.6504359662532808 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 1 1429
9 3
9 25 17
checkcorrect 2 2 real score 1.4635172322392465 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 2 1429
9 140
9 67 78
checkcorrect 2 2 real score 1.6883579045534134 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 3 1429
0 1
9 11 32
checkcorrect 2 2 real score 0.7167330622673035 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 4 1429
9 48
9 62 38
checkcorrect 2 2 real score 1.513030433654785 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 5 1429
9 150
9 11 44
checkcorrect 4 4 real score 1.268584681302309 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 tot

9 42
9 34 37
checkcorrect 2 2 real score 1.446180707961321 Hits@1 0.8076923076923077 Hits@3 1.0 Hits@10 1.0 MRR 0.8910256410256411 cur_rank 0 abs_cur_rank 0 total_num 51 1429
9 80
9 67 41
checkcorrect 2 2 real score 1.7618943929672242 Hits@1 0.8113207547169812 Hits@3 1.0 Hits@10 1.0 MRR 0.8930817610062893 cur_rank 0 abs_cur_rank 0 total_num 52 1429
9 25
9 38 34
checkcorrect 2 2 real score 1.6037633210420608 Hits@1 0.8148148148148148 Hits@3 1.0 Hits@10 1.0 MRR 0.8950617283950618 cur_rank 0 abs_cur_rank 0 total_num 53 1429
9 13
9 36 26
checkcorrect 2 2 real score 1.652468004822731 Hits@1 0.8181818181818182 Hits@3 1.0 Hits@10 1.0 MRR 0.896969696969697 cur_rank 0 abs_cur_rank 0 total_num 54 1429
0 1
9 45 69
checkcorrect 2 2 real score 0.5779496625065803 Hits@1 0.8214285714285714 Hits@3 1.0 Hits@10 1.0 MRR 0.8988095238095238 cur_rank 0 abs_cur_rank 0 total_num 55 1429
9 11
9 28 38
checkcorrect 2 2 real score 1.564708411693573 Hits@1 0.8245614035087719 Hits@3 1.0 Hits@10 1.0 MRR 0.9005847953

9 15
9 44 38
checkcorrect 2 2 real score 1.6520869851112365 Hits@1 0.82 Hits@3 1.0 Hits@10 1.0 MRR 0.9 cur_rank 0 abs_cur_rank 0 total_num 99 1429
9 109
9 71 50
checkcorrect 12 12 real score 0.9853025266900659 Hits@1 0.8217821782178217 Hits@3 1.0 Hits@10 1.0 MRR 0.900990099009901 cur_rank 0 abs_cur_rank 0 total_num 100 1429
0 0
9 9 10
checkcorrect 2 2 real score 0.6270277172327041 Hits@1 0.8137254901960784 Hits@3 1.0 Hits@10 1.0 MRR 0.8970588235294118 cur_rank 1 abs_cur_rank 1 total_num 101 1429
0 1
9 24 33
checkcorrect 2 2 real score 0.7370968997478485 Hits@1 0.8155339805825242 Hits@3 1.0 Hits@10 1.0 MRR 0.8980582524271845 cur_rank 0 abs_cur_rank 0 total_num 102 1429
9 128
9 34 33
checkcorrect 2 2 real score 1.6770299941301348 Hits@1 0.8173076923076923 Hits@3 1.0 Hits@10 1.0 MRR 0.8990384615384616 cur_rank 0 abs_cur_rank 0 total_num 103 1429
9 32
9 38 13
checkcorrect 4 4 real score 1.7619756221771241 Hits@1 0.819047619047619 Hits@3 1.0 Hits@10 1.0 MRR 0.9 cur_rank 0 abs_cur_rank 0 tot

9 7 24
checkcorrect 2 2 real score 0.5770882606506348 Hits@1 0.8108108108108109 Hits@3 1.0 Hits@10 1.0 MRR 0.8952702702702703 cur_rank 0 abs_cur_rank 0 total_num 147 1429
9 127
9 78 73
checkcorrect 2 2 real score 1.695519083738327 Hits@1 0.8120805369127517 Hits@3 1.0 Hits@10 1.0 MRR 0.8959731543624161 cur_rank 0 abs_cur_rank 0 total_num 148 1429
9 40
9 39 9
checkcorrect 4 4 real score 1.4162578895688056 Hits@1 0.8133333333333334 Hits@3 1.0 Hits@10 1.0 MRR 0.8966666666666666 cur_rank 0 abs_cur_rank 0 total_num 149 1429
9 46
9 33 17
checkcorrect 2 2 real score 1.3283486366271973 Hits@1 0.8145695364238411 Hits@3 1.0 Hits@10 1.0 MRR 0.8973509933774835 cur_rank 0 abs_cur_rank 0 total_num 150 1429
9 10
9 6 11
checkcorrect 12 12 real score 1.6526967167854307 Hits@1 0.8157894736842105 Hits@3 1.0 Hits@10 1.0 MRR 0.8980263157894737 cur_rank 0 abs_cur_rank 0 total_num 151 1429
9 3
9 21 17
checkcorrect 2 2 real score 1.7367217257618903 Hits@1 0.8169934640522876 Hits@3 1.0 Hits@10 1.0 MRR 0.8986928

9 3
9 11 19
checkcorrect 2 2 real score 1.7140884041786193 Hits@1 0.8256410256410256 Hits@3 1.0 Hits@10 1.0 MRR 0.9042735042735043 cur_rank 0 abs_cur_rank 0 total_num 194 1429
9 9
9 19 28
checkcorrect 2 2 real score 1.7007254481315615 Hits@1 0.826530612244898 Hits@3 1.0 Hits@10 1.0 MRR 0.9047619047619048 cur_rank 0 abs_cur_rank 0 total_num 195 1429
0 1
9 14 9
checkcorrect 2 2 real score 0.6149253867566585 Hits@1 0.8274111675126904 Hits@3 1.0 Hits@10 1.0 MRR 0.9052453468697124 cur_rank 0 abs_cur_rank 0 total_num 196 1429
9 3
9 28 29
checkcorrect 2 2 real score 1.616503268480301 Hits@1 0.8282828282828283 Hits@3 1.0 Hits@10 1.0 MRR 0.9057239057239058 cur_rank 0 abs_cur_rank 0 total_num 197 1429
0 0
9 20 27
checkcorrect 6 6 real score 0.05579934511333704 Hits@1 0.8241206030150754 Hits@3 1.0 Hits@10 1.0 MRR 0.9036850921273032 cur_rank 1 abs_cur_rank 1 total_num 198 1429
9 80
9 48 56
checkcorrect 2 2 real score 1.6802971482276918 Hits@1 0.825 Hits@3 1.0 Hits@10 1.0 MRR 0.9041666666666667 cur

...

9 11 5
checkcorrect 4 4 real score 0.31053894609212873 Hits@1 0.8287781350482315 Hits@3 0.9855305466237942 Hits@10 1.0 MRR 0.9039108355024758 cur_rank 0 abs_cur_rank 0 total_num 1243 1429
0 1
9 13 29
checkcorrect 2 2 real score 0.6579370856285095 Hits@1 0.8289156626506025 Hits@3 0.9855421686746988 Hits@10 1.0 MRR 0.903988015554281 cur_rank 0 abs_cur_rank 0 total_num 1244 1429
9 8
9 38 14
checkcorrect 4 4 real score 1.5669877529144287 Hits@1 0.8290529695024077 Hits@3 0.985553772070626 Hits@10 1.0 MRR 0.9040650717215729 cur_rank 0 abs_cur_rank 0 total_num 1245 1429
9 86
9 55 14
checkcorrect 2 2 real score 1.6540729284286497 Hits@1 0.8291900561347233 Hits@3 0.9855653568564555 Hits@10 1.0 MRR 0.9041420043023897 cur_rank 0 abs_cur_rank 0 total_num 1246 1429
0 1
9 9 22
checkcorrect 2 2 real score 0.7127049088478088 Hits@1 0.8293269230769231 Hits@3 0.9855769230769231 Hits@10 1.0 MRR 0.904218813593814 cur_rank 0 abs_cur_rank 0 total_num 1247 1429
0 0
9 28 21
checkcorrect 4 4 real score 0.53843

9 10 27
checkcorrect 2 2 real score 0.5283653765916825 Hits@1 0.8275058275058275 Hits@3 0.9852369852369852 Hits@10 1.0 MRR 0.903301019967687 cur_rank 0 abs_cur_rank 0 total_num 1286 1429
0 0
9 22 34
checkcorrect 4 4 real score 0.4582273781299591 Hits@1 0.8268633540372671 Hits@3 0.985248447204969 Hits@10 1.0 MRR 0.9029878980577741 cur_rank 1 abs_cur_rank 1 total_num 1287 1429
0 1
9 14 7
checkcorrect 2 2 real score 0.6899787485599518 Hits@1 0.8269976726144298 Hits@3 0.9852598913886734 Hits@10 1.0 MRR 0.9030631595798395 cur_rank 0 abs_cur_rank 0 total_num 1288 1429
9 15
9 37 27
checkcorrect 2 2 real score 1.6995275110006332 Hits@1 0.8271317829457364 Hits@3 0.9852713178294573 Hits@10 1.0 MRR 0.9031383044173745 cur_rank 0 abs_cur_rank 0 total_num 1289 1429
0 1
9 53 20
checkcorrect 2 2 real score 0.5228402700275183 Hits@1 0.8272656855151046 Hits@3 0.9852827265685515 Hits@10 1.0 MRR 0.9032133328415284 cur_rank 0 abs_cur_rank 0 total_num 1290 1429
9 84
9 31 25
checkcorrect 6 6 real score 0.883

0 1
9 27 23
checkcorrect 2 2 real score 0.6193193018436431 Hits@1 0.8263157894736842 Hits@3 0.9857142857142858 Hits@10 1.0 MRR 0.9025426661892828 cur_rank 0 abs_cur_rank 0 total_num 1329 1429
0 0
9 17 14
checkcorrect 12 12 real score 0.6514402568340302 Hits@1 0.8264462809917356 Hits@3 0.98572501878287 Hits@10 1.0 MRR 0.9026158873266312 cur_rank 0 abs_cur_rank 0 total_num 1330 1429
0 1
9 22 27
checkcorrect 2 2 real score 0.6130239427089691 Hits@1 0.8258258258258259 Hits@3 0.9857357357357357 Hits@10 1.0 MRR 0.9023136231469566 cur_rank 1 abs_cur_rank 1 total_num 1331 1429
0 0
0 1 1
checkcorrect 4 4 real score 0.0 Hits@1 0.8252063015753939 Hits@3 0.9857464366091523 Hits@10 1.0 MRR 0.9018867812191144 cur_rank 2 abs_cur_rank 2 total_num 1332 1429
9 116
9 30 39
checkcorrect 4 4 real score 1.2457169443368912 Hits@1 0.8253373313343328 Hits@3 0.9857571214392804 Hits@10 1.0 MRR 0.90196032935913 cur_rank 0 abs_cur_rank 0 total_num 1333 1429
9 16
9 22 19
checkcorrect 4 4 real score 1.31413736939430

9 9 17
checkcorrect 4 4 real score 0.5546155512332916 Hits@1 0.8244719592134013 Hits@3 0.9847050254916242 Hits@10 1.0 MRR 0.9011641753083851 cur_rank 0 abs_cur_rank 0 total_num 1372 1429
9 110
9 22 34
checkcorrect 2 2 real score 1.724886918067932 Hits@1 0.8245997088791849 Hits@3 0.9847161572052402 Hits@10 1.0 MRR 0.9012361082230078 cur_rank 0 abs_cur_rank 0 total_num 1373 1429
0 1
9 6 7
checkcorrect 2 2 real score 0.6253774434328079 Hits@1 0.8247272727272728 Hits@3 0.9847272727272727 Hits@10 1.0 MRR 0.9013079365079365 cur_rank 0 abs_cur_rank 0 total_num 1374 1429
0 1
9 7 10
checkcorrect 2 2 real score 0.739851713180542 Hits@1 0.8248546511627907 Hits@3 0.9847383720930233 Hits@10 1.0 MRR 0.9013796603912883 cur_rank 0 abs_cur_rank 0 total_num 1375 1429
9 72
9 13 25
checkcorrect 4 4 real score 0.7654629476368427 Hits@1 0.8242556281771968 Hits@3 0.9847494553376906 Hits@10 1.0 MRR 0.9010881718942721 cur_rank 1 abs_cur_rank 1 total_num 1376 1429
0 1
9 11 36
checkcorrect 2 2 real score 0.77321

9 16 29
checkcorrect 2 2 real score 0.6077509105205536 Hits@1 0.8248587570621468 Hits@3 0.9844632768361582 Hits@10 1.0 MRR 0.9013995381580127 cur_rank 0 abs_cur_rank 0 total_num 1415 1429
9 26
9 57 22
checkcorrect 6 6 real score 0.7021024256013334 Hits@1 0.8242766407904023 Hits@3 0.9844742413549753 Hits@10 1.0 MRR 0.9011162639603005 cur_rank 1 abs_cur_rank 1 total_num 1416 1429
9 76
9 49 25
checkcorrect 2 2 real score 1.7001095950603484 Hits@1 0.8244005641748943 Hits@3 0.9844851904090268 Hits@10 1.0 MRR 0.9011859986119506 cur_rank 0 abs_cur_rank 0 total_num 1417 1429
0 1
9 42 3
checkcorrect 4 4 real score 0.5891951829195022 Hits@1 0.8245243128964059 Hits@3 0.9844961240310077 Hits@10 1.0 MRR 0.9012556349765651 cur_rank 0 abs_cur_rank 0 total_num 1418 1429
0 0
9 24 31
checkcorrect 4 4 real score 0.2893545016646385 Hits@1 0.823943661971831 Hits@3 0.9845070422535211 Hits@10 1.0 MRR 0.9009730605857366 cur_rank 1 abs_cur_rank 1 total_num 1419 1429
0 0
9 11 22
checkcorrect 4 4 real score 0.46

In [32]:
###########################################
##obtain the AUC-PR for the test triples###
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
import matplotlib.pyplot as plt

#we select all the triples in the inductive test set
pos_triples = list(data_ind_test)

#we build the negative samples by randomly replace head or tail entity in the triple.
neg_triples = list()

for i in range(len(pos_triples)):
    
    s_pos, r_pos, t_pos = pos_triples[i][0], pos_triples[i][1], pos_triples[i][2]
    
    #decide to replace the head or tail entity
    number_0 = random.uniform(0, 1)
    
    if number_0 < 0.5: #replace head entity
        
        s_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_neg, r_pos, t_pos) in data_test) or (
               (s_neg, r_pos, t_pos) in data_valid) or (
               (s_neg, r_pos, t_pos) in data) or (
               (s_neg, r_pos, t_pos) in data_ind) or (
               (s_neg, r_pos, t_pos) in data_ind_valid) or (
               (s_neg, r_pos, t_pos) in data_ind_test):
            
            s_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_neg, r_pos, t_pos))
    
    else: #replace tail entity

        t_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_pos, r_pos, t_neg) in data_test) or (
               (s_pos, r_pos, t_neg) in data_valid) or (
               (s_pos, r_pos, t_neg) in data) or (
               (s_pos, r_pos, t_neg) in data_ind) or (
               (s_pos, r_pos, t_neg) in data_ind_valid) or (
               (s_pos, r_pos, t_neg) in data_ind_test):
            
            t_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_pos, r_pos, t_neg))

if len(pos_triples) != len(neg_triples):
    raise ValueError('error when generating negative triples')
        
#combine all triples
all_triples = pos_triples + neg_triples

#obtain the label array
arr1 = np.ones((len(pos_triples),))
arr2 = np.zeros((len(neg_triples),))
y_test = np.concatenate((arr1, arr2))

#shuffle positive and negative triples (optional)
all_triples, y_test = shuffle(all_triples, y_test)

#obtain the score aray
y_score = np.zeros((len(y_test),))

#implement the scoring
for i in range(len(all_triples)):
    
    s, r, t = all_triples[i][0], all_triples[i][1], all_triples[i][2]
    
    #path_score = path_based_triple_scoring(s, r, t, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)
    
    subg_score = subgraph_triple_scoring(s, r, t, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #ave_score = (path_score + subg_score)/float(2)
    
    #y_score[i] = ave_score
    y_score[i] = subg_score
    
    if i % 20 == 0 and i > 0:
        print('evaluating scores', i, len(all_triples))
        
        # Data to plot precision - recall curve
        precision, recall, thresholds = precision_recall_curve(y_test[:i], y_score[:i])
        # Use AUC function to calculate the area under the curve of precision recall curve
        auc_precision_recall = auc(recall, precision)
        print('AUC-PR is:', auc_precision_recall)
        
        
# Data to plot precision - recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_score)
# Use AUC function to calculate the area under the curve of precision recall curve
auc_precision_recall = auc(recall, precision)
print('AUC-PR is:', auc_precision_recall)

evaluating scores 20 2858
AUC-PR is: 0.6216331789125906
evaluating scores 40 2858
AUC-PR is: 0.7616961827804294
evaluating scores 60 2858
AUC-PR is: 0.7401220289707974
evaluating scores 80 2858
AUC-PR is: 0.7535261091294837
evaluating scores 100 2858
AUC-PR is: 0.7608552270081776
evaluating scores 120 2858
AUC-PR is: 0.7607503556214654
evaluating scores 140 2858
AUC-PR is: 0.7799555582077222
evaluating scores 160 2858
AUC-PR is: 0.7747786548366198
evaluating scores 180 2858
AUC-PR is: 0.7849838527038903
evaluating scores 200 2858
AUC-PR is: 0.753651240516782
evaluating scores 220 2858
AUC-PR is: 0.7564995239384278
evaluating scores 240 2858
AUC-PR is: 0.7485437788549277
evaluating scores 260 2858
AUC-PR is: 0.7501146433823228
evaluating scores 280 2858
AUC-PR is: 0.7349880426463673
evaluating scores 300 2858
AUC-PR is: 0.7554593136175546
evaluating scores 320 2858
AUC-PR is: 0.7367091438244133
evaluating scores 340 2858
AUC-PR is: 0.7327606077333607
evaluating scores 360 2858
AUC-PR is

In [33]:
##########################################################
##obtain the AUC-PR for the test triples, using sklearn###
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
import matplotlib.pyplot as plt

#we select all the triples in the inductive test set
pos_triples = list(data_ind_test)

#we build the negative samples by randomly replace head or tail entity in the triple.
neg_triples = list()

for i in range(len(pos_triples)):
    
    s_pos, r_pos, t_pos = pos_triples[i][0], pos_triples[i][1], pos_triples[i][2]
    
    #decide to replace the head or tail entity
    number_0 = random.uniform(0, 1)
    
    if number_0 < 0.5: #replace head entity
        
        s_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_neg, r_pos, t_pos) in data_test) or (
               (s_neg, r_pos, t_pos) in data_valid) or (
               (s_neg, r_pos, t_pos) in data) or (
               (s_neg, r_pos, t_pos) in data_ind) or (
               (s_neg, r_pos, t_pos) in data_ind_valid) or (
               (s_neg, r_pos, t_pos) in data_ind_test):
            
            s_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_neg, r_pos, t_pos))
    
    else: #replace tail entity

        t_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_pos, r_pos, t_neg) in data_test) or (
               (s_pos, r_pos, t_neg) in data_valid) or (
               (s_pos, r_pos, t_neg) in data) or (
               (s_pos, r_pos, t_neg) in data_ind) or (
               (s_pos, r_pos, t_neg) in data_ind_valid) or (
               (s_pos, r_pos, t_neg) in data_ind_test):
            
            t_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_pos, r_pos, t_neg))

if len(pos_triples) != len(neg_triples):
    raise ValueError('error when generating negative triples')
        
#combine all triples
all_triples = pos_triples + neg_triples

#obtain the label array
arr1 = np.ones((len(pos_triples),))
arr2 = np.zeros((len(neg_triples),))
y_test = np.concatenate((arr1, arr2))

#shuffle positive and negative triples (optional)
all_triples, y_test = shuffle(all_triples, y_test)

#obtain the score aray
y_score = np.zeros((len(y_test),))

#implement the scoring
for i in range(len(all_triples)):
    
    s, r, t = all_triples[i][0], all_triples[i][1], all_triples[i][2]
    
    #path_score = path_based_triple_scoring(s, r, t, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)
    
    subg_score = subgraph_triple_scoring(s, r, t, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #ave_score = (path_score + subg_score)/float(2)
    
    #y_score[i] = ave_score
    y_score[i] = subg_score
    
    if i % 20 == 0 and i > 0:
        print('evaluating scores', i, len(all_triples))
        auc = metrics.roc_auc_score(y_test[:i], y_score[:i])
        auc_pr = metrics.average_precision_score(y_test[:i], y_score[:i])
        print('auc, auc-pr', auc, auc_pr)
        
print('evaluating scores', i, len(all_triples))
auc = metrics.roc_auc_score(y_test, y_score)
auc_pr = metrics.average_precision_score(y_test, y_score)
print('(final) auc, auc-pr', auc, auc_pr)

evaluating scores 20 2858
auc, auc-pr 0.8273809523809523 0.9223897134611418
evaluating scores 40 2858
auc, auc-pr 0.7129120879120879 0.8261535330653095
evaluating scores 60 2858
auc, auc-pr 0.6827262044653349 0.7666396582155786
evaluating scores 80 2858
auc, auc-pr 0.6919025674786045 0.771587898576326
evaluating scores 100 2858
auc, auc-pr 0.6293103448275862 0.7008871520605896
evaluating scores 120 2858
auc, auc-pr 0.6230203619909502 0.6972042345057767
evaluating scores 140 2858
auc, auc-pr 0.6346516007532956 0.715588033051898
evaluating scores 160 2858
auc, auc-pr 0.6591232789998417 0.7087285574477529
evaluating scores 180 2858
auc, auc-pr 0.6514718668488386 0.6935327749467828
evaluating scores 200 2858
auc, auc-pr 0.6650660264105642 0.686640910821871
evaluating scores 220 2858
auc, auc-pr 0.6759351208209201 0.701527797912207
evaluating scores 240 2858
auc, auc-pr 0.6711624026696329 0.6986142163498967
evaluating scores 260 2858
auc, auc-pr 0.6738499792789059 0.6936482332482707
evaluat

evaluating scores 2160 2858
auc, auc-pr 0.719288437843775 0.7170113227028303
evaluating scores 2180 2858
auc, auc-pr 0.7192394781144781 0.7172964696308375
evaluating scores 2200 2858
auc, auc-pr 0.7197393293048314 0.7183175457222959
evaluating scores 2220 2858
auc, auc-pr 0.7195028995765905 0.7191755824482717
evaluating scores 2240 2858
auc, auc-pr 0.7194029041527534 0.7193302150791707
evaluating scores 2260 2858
auc, auc-pr 0.7195094442477654 0.7188444384448561
evaluating scores 2280 2858
auc, auc-pr 0.7196926614435478 0.7186189205875941
evaluating scores 2300 2858
auc, auc-pr 0.7184649786483299 0.7176700660358429
evaluating scores 2320 2858
auc, auc-pr 0.7187434738631238 0.715841129068929
evaluating scores 2340 2858
auc, auc-pr 0.7184554714684702 0.7161894233814056
evaluating scores 2360 2858
auc, auc-pr 0.7199523650822839 0.7185213176121503
evaluating scores 2380 2858
auc, auc-pr 0.7172910033927609 0.715787763838704
evaluating scores 2400 2858
auc, auc-pr 0.7177772576235775 0.717019

In [34]:
######################################################
#obtain the Hits@N for entity prediction##############

#we select all the triples in the inductive test set
selected = list(data_ind_test)

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0
Hits_at_3 = 0
Hits_at_10 = 0
MRR_raw = 0.

for i in range(len(selected)):
    
    triple_list = list()
    
    #score the true triple
    s_pos, r_pos, t_pos = selected[i][0], selected[i][1], selected[i][2]

    #path_score = path_based_triple_scoring(s_pos, r_pos, t_pos, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

    subg_score = subgraph_triple_scoring(s_pos, r_pos, t_pos, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #ave_score = (path_score + subg_score)/float(2)
    
    triple_list.append([(s_pos, r_pos, t_pos), subg_score])
    
    #generate the 50 random samples
    for sub_i in range(50):
        
        #decide to replace the head or tail entity
        number_0 = random.uniform(0, 1)

        if number_0 < 0.5: #replace head entity
            
            s_neg = random.choice(list(new_ent_set))
            
            while ((s_neg, r_pos, t_pos) in data_test) or (
                   (s_neg, r_pos, t_pos) in data_valid) or (
                   (s_neg, r_pos, t_pos) in data) or (
                   (s_neg, r_pos, t_pos) in data_ind) or (
                   (s_neg, r_pos, t_pos) in data_ind_valid) or (
                   (s_neg, r_pos, t_pos) in data_ind_test):

                s_neg = random.choice(list(new_ent_set))
            
            #path_score = path_based_triple_scoring(s_neg, r_pos, t_pos, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

            subg_score = subgraph_triple_scoring(s_neg, r_pos, t_pos, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)

            #ave_score = (path_score + subg_score)/float(2)

            triple_list.append([(s_neg, r_pos, t_pos), subg_score])
            
        else: #replace tail entity

            t_neg = random.choice(list(new_ent_set))
            
            #filter out the existing triples
            while ((s_pos, r_pos, t_neg) in data_test) or (
                   (s_pos, r_pos, t_neg) in data_valid) or (
                   (s_pos, r_pos, t_neg) in data) or (
                   (s_pos, r_pos, t_neg) in data_ind) or (
                   (s_pos, r_pos, t_neg) in data_ind_valid) or (
                   (s_pos, r_pos, t_neg) in data_ind_test):

                t_neg = random.choice(list(new_ent_set))
            
            #path_score = path_based_triple_scoring(s_pos, r_pos, t_neg, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

            subg_score = subgraph_triple_scoring(s_pos, r_pos, t_neg, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)

            #ave_score = (path_score + subg_score)/float(2)

            triple_list.append([(s_pos, r_pos, t_neg), subg_score])
            
    #random shuffle!
    random.shuffle(triple_list)
    
    #sort
    sorted_list = sorted(triple_list, key = lambda x: x[-1], reverse=True)
    
    p = 0
    
    while p < len(sorted_list) and sorted_list[p][0] != (s_pos, r_pos, t_pos):
            
        p += 1
    
    if p == 0:
        
        Hits_at_1 += 1
        
    if p < 3:
        
        Hits_at_3 += 1
        
    if p < 10:
        
        Hits_at_10 += 1
        
    MRR_raw += 1./float(p + 1.) 
        
    print('checkcorrect', (s_pos, r_pos, t_pos), sorted_list[p][0],
          'real score', sorted_list[p][-1],
          'Hits@1', Hits_at_1/(i+1),
          'Hits@3', Hits_at_3/(i+1),
          'Hits@10', Hits_at_10/(i+1),
          'MRR', MRR_raw/(i+1),
          'rank', p,
          'total_num', i, len(selected))

checkcorrect (9272, 2, 3997) (9272, 2, 3997) real score 0.6795412719249725 Hits@1 0.0 Hits@3 0.0 Hits@10 0.0 MRR 0.08333333333333333 rank 11 total_num 0 1429
checkcorrect (6007, 2, 5903) (6007, 2, 5903) real score 0.7114194571971894 Hits@1 0.0 Hits@3 0.0 Hits@10 0.5 MRR 0.125 rank 5 total_num 1 1429
checkcorrect (10413, 2, 7851) (10413, 2, 7851) real score 0.6482020348310471 Hits@1 0.0 Hits@3 0.0 Hits@10 0.6666666666666666 MRR 0.13888888888888887 rank 5 total_num 2 1429
checkcorrect (4566, 2, 7229) (4566, 2, 7229) real score 0.7900169491767883 Hits@1 0.25 Hits@3 0.25 Hits@10 0.75 MRR 0.35416666666666663 rank 0 total_num 3 1429
checkcorrect (9752, 2, 7279) (9752, 2, 7279) real score 0.704738998413086 Hits@1 0.2 Hits@3 0.2 Hits@10 0.8 MRR 0.3055555555555555 rank 8 total_num 4 1429
checkcorrect (8071, 2, 5711) (8071, 2, 5711) real score 0.6707434564828872 Hits@1 0.16666666666666666 Hits@3 0.16666666666666666 Hits@10 0.8333333333333334 MRR 0.2962962962962963 rank 3 total_num 5 1429
checkco

checkcorrect (6621, 4, 4577) (6621, 4, 4577) real score 0.13498107641935347 Hits@1 0.1111111111111111 Hits@3 0.24444444444444444 Hits@10 0.5555555555555556 MRR 0.25040541131805405 rank 39 total_num 44 1429
checkcorrect (9275, 2, 10054) (9275, 2, 10054) real score 0.6295775234699249 Hits@1 0.10869565217391304 Hits@3 0.2391304347826087 Hits@10 0.5434782608695652 MRR 0.2464110907821543 rank 14 total_num 45 1429
checkcorrect (3991, 2, 3990) (3991, 2, 3990) real score 0.7001687407493591 Hits@1 0.10638297872340426 Hits@3 0.23404255319148937 Hits@10 0.5319148936170213 MRR 0.24294135126196664 rank 11 total_num 46 1429
checkcorrect (9550, 2, 4354) (9550, 2, 4354) real score 0.6470039546489715 Hits@1 0.10416666666666667 Hits@3 0.22916666666666666 Hits@10 0.5208333333333334 MRR 0.23910556330675412 rank 16 total_num 47 1429
checkcorrect (5734, 0, 9917) (5734, 0, 9917) real score 0.8032101899385452 Hits@1 0.12244897959183673 Hits@3 0.24489795918367346 Hits@10 0.5306122448979592 MRR 0.25463402119845

checkcorrect (10390, 4, 5892) (10390, 4, 5892) real score 0.48198080956935885 Hits@1 0.12790697674418605 Hits@3 0.2558139534883721 Hits@10 0.5 MRR 0.25651746148068294 rank 24 total_num 85 1429
checkcorrect (8877, 2, 5344) (8877, 2, 5344) real score 0.7354412257671357 Hits@1 0.13793103448275862 Hits@3 0.26436781609195403 Hits@10 0.5057471264367817 MRR 0.26506323778550267 rank 0 total_num 86 1429
checkcorrect (6572, 2, 9885) (6572, 2, 9885) real score 0.6535520732402802 Hits@1 0.13636363636363635 Hits@3 0.26136363636363635 Hits@10 0.5 MRR 0.2628628438496285 rank 13 total_num 87 1429
checkcorrect (4147, 4, 4457) (4147, 4, 4457) real score 0.5925023019313812 Hits@1 0.1348314606741573 Hits@3 0.25842696629213485 Hits@10 0.5056179775280899 MRR 0.26151446518679156 rank 6 total_num 88 1429
checkcorrect (10788, 4, 10428) (10788, 4, 10428) real score 0.6791635990142822 Hits@1 0.13333333333333333 Hits@3 0.26666666666666666 Hits@10 0.5111111111111111 MRR 0.26416430446249384 rank 1 total_num 89 1429

checkcorrect (7596, 2, 4073) (7596, 2, 4073) real score 0.5430383712053299 Hits@1 0.11023622047244094 Hits@3 0.2440944881889764 Hits@10 0.5118110236220472 MRR 0.24098088768408735 rank 19 total_num 126 1429
checkcorrect (7853, 2, 6543) (7853, 2, 6543) real score 0.6825966894626617 Hits@1 0.109375 Hits@3 0.25 Hits@10 0.515625 MRR 0.2430044744990554 rank 1 total_num 127 1429
checkcorrect (8266, 8, 4309) (8266, 8, 4309) real score 0.9362723469734192 Hits@1 0.10852713178294573 Hits@3 0.2558139534883721 Hits@10 0.5193798449612403 MRR 0.24499668787503173 rank 1 total_num 128 1429
checkcorrect (8854, 2, 7772) (8854, 2, 7772) real score 0.4339876502752304 Hits@1 0.1076923076923077 Hits@3 0.25384615384615383 Hits@10 0.5153846153846153 MRR 0.24336023692611708 rank 30 total_num 129 1429
checkcorrect (5293, 2, 5084) (5293, 2, 5084) real score 0.19599015936255454 Hits@1 0.10687022900763359 Hits@3 0.25190839694656486 Hits@10 0.5114503816793893 MRR 0.24176575235890008 rank 28 total_num 130 1429
checkc

checkcorrect (6324, 2, 6323) (6324, 2, 6323) real score 0.7366640031337738 Hits@1 0.10714285714285714 Hits@3 0.25 Hits@10 0.5238095238095238 MRR 0.24051394358611575 rank 5 total_num 167 1429
checkcorrect (10521, 2, 6496) (10521, 2, 6496) real score 0.6581735104322434 Hits@1 0.10650887573964497 Hits@3 0.2485207100591716 Hits@10 0.5207100591715976 MRR 0.23954595029225165 rank 12 total_num 168 1429
checkcorrect (4862, 2, 5601) (4862, 2, 5601) real score 0.5908470302820206 Hits@1 0.10588235294117647 Hits@3 0.24705882352941178 Hits@10 0.5176470588235295 MRR 0.23843097411406192 rank 19 total_num 169 1429
checkcorrect (6880, 2, 5789) (6880, 2, 5789) real score 0.6827942550182342 Hits@1 0.10526315789473684 Hits@3 0.24561403508771928 Hits@10 0.52046783625731 MRR 0.2384986292362019 rank 3 total_num 170 1429
checkcorrect (10314, 4, 6400) (10314, 4, 6400) real score 0.5612967997789383 Hits@1 0.10465116279069768 Hits@3 0.2441860465116279 Hits@10 0.5174418604651163 MRR 0.23745400656280402 rank 16 to

...

checkcorrect (4049, 6, 10567) (4049, 6, 10567) real score 0.7744194626808166 Hits@1 0.09213661636219221 Hits@3 0.24384432088959493 Hits@10 0.5138999205718825 MRR 0.228025200404693 rank 5 total_num 1258 1429
checkcorrect (4771, 0, 8448) (4771, 0, 8448) real score 0.10710040926933288 Hits@1 0.09206349206349207 Hits@3 0.24365079365079365 Hits@10 0.5134920634920634 MRR 0.2279163780955695 rank 10 total_num 1259 1429
checkcorrect (8875, 2, 4898) (8875, 2, 4898) real score 0.666339322924614 Hits@1 0.09199048374306107 Hits@3 0.24345757335448057 Hits@10 0.5138778747026169 MRR 0.2279338908805849 rank 3 total_num 1260 1429
checkcorrect (3939, 2, 5453) (3939, 2, 5453) real score 0.7605455338954925 Hits@1 0.0919175911251981 Hits@3 0.24405705229793978 Hits@10 0.5142630744849446 MRR 0.2281494741683182 rank 1 total_num 1261 1429
checkcorrect (10552, 4, 7949) (10552, 4, 7949) real score 0.8780369877815246 Hits@1 0.09184481393507522 Hits@3 0.24465558194774348 Hits@10 0.5146476642913698 MRR 0.22836471607

checkcorrect (4289, 2, 4327) (4289, 2, 4327) real score 0.6782160639762879 Hits@1 0.09160892994611239 Hits@3 0.24403387220939185 Hits@10 0.5173210161662818 MRR 0.2287501954939034 rank 5 total_num 1298 1429
checkcorrect (8241, 2, 8240) (8241, 2, 8240) real score 0.7395528078079223 Hits@1 0.09153846153846154 Hits@3 0.24384615384615385 Hits@10 0.5176923076923077 MRR 0.22868412391495208 rank 6 total_num 1299 1429
checkcorrect (4310, 2, 6195) (4310, 2, 6195) real score 0.6044537723064423 Hits@1 0.09146810146041506 Hits@3 0.2436587240584166 Hits@10 0.5172943889315911 MRR 0.22855105045733531 rank 17 total_num 1300 1429
checkcorrect (8080, 4, 6009) (8080, 4, 6009) real score 0.7002768278121948 Hits@1 0.0913978494623656 Hits@3 0.2434715821812596 Hits@10 0.5176651305683564 MRR 0.22847151816051708 rank 7 total_num 1301 1429
checkcorrect (4229, 4, 4549) (4229, 4, 4549) real score 0.445256832242012 Hits@1 0.09132770529547199 Hits@3 0.24328472755180353 Hits@10 0.5172678434382195 MRR 0.22836013045151

checkcorrect (4146, 2, 7624) (4146, 2, 7624) real score 0.5715389430522919 Hits@1 0.09185959671396565 Hits@3 0.24421209858103063 Hits@10 0.5175504107542942 MRR 0.22879144415157618 rank 13 total_num 1338 1429
checkcorrect (4155, 2, 8485) (4155, 2, 8485) real score 0.5351554471999407 Hits@1 0.09179104477611941 Hits@3 0.24402985074626865 Hits@10 0.5171641791044777 MRR 0.22866216363769853 rank 17 total_num 1339 1429
checkcorrect (7104, 2, 8712) (7104, 2, 8712) real score 0.7172119498252869 Hits@1 0.09172259507829977 Hits@3 0.24384787472035793 Hits@10 0.517524235645041 MRR 0.2285981777907332 rank 6 total_num 1340 1429
checkcorrect (7492, 4, 5407) (7492, 4, 5407) real score 0.7157328486442566 Hits@1 0.09165424739195231 Hits@3 0.2436661698956781 Hits@10 0.5178837555886736 MRR 0.22855202912372571 rank 5 total_num 1341 1429
checkcorrect (5195, 6, 6773) (5195, 6, 6773) real score 0.4216143973171711 Hits@1 0.09158600148920328 Hits@3 0.24348473566641846 Hits@10 0.5182427401340283 MRR 0.22853076923

checkcorrect (8303, 2, 8871) (8303, 2, 8871) real score 0.5281083792448044 Hits@1 0.09137055837563451 Hits@3 0.242929659173314 Hits@10 0.5155910079767948 MRR 0.22782551599954895 rank 24 total_num 1378 1429
checkcorrect (5349, 2, 5348) (5349, 2, 5348) real score 0.6371071636676788 Hits@1 0.09130434782608696 Hits@3 0.2427536231884058 Hits@10 0.5152173913043478 MRR 0.22769856387125026 rank 18 total_num 1379 1429
checkcorrect (9416, 8, 7216) (9416, 8, 7216) real score 0.6353716000914573 Hits@1 0.09123823316437364 Hits@3 0.2433019551049964 Hits@10 0.5155684286748733 MRR 0.2277750553770157 rank 2 total_num 1380 1429
checkcorrect (8035, 4, 10140) (8035, 4, 10140) real score 0.6932038009166718 Hits@1 0.09117221418234443 Hits@3 0.24312590448625182 Hits@10 0.515918958031838 MRR 0.2276906386300794 rank 8 total_num 1381 1429
checkcorrect (6392, 2, 8676) (6392, 2, 8676) real score 0.7151348471641541 Hits@1 0.0911062906724512 Hits@3 0.24295010845986983 Hits@10 0.5162689804772235 MRR 0.22762929843067

checkcorrect (6337, 4, 7629) (6337, 4, 7629) real score 0.6121881663799286 Hits@1 0.0923185341789993 Hits@3 0.2459478505990134 Hits@10 0.5200845665961945 MRR 0.2295552872711329 rank 6 total_num 1418 1429
checkcorrect (7152, 4, 3960) (7152, 4, 3960) real score 0.2718981817364693 Hits@1 0.09225352112676057 Hits@3 0.24577464788732395 Hits@10 0.5197183098591549 MRR 0.22941496878031026 rank 32 total_num 1419 1429
checkcorrect (10384, 4, 6155) (10384, 4, 6155) real score 0.4668690234422684 Hits@1 0.09218859957776214 Hits@3 0.24560168895144266 Hits@10 0.5193525686136523 MRR 0.22927865584359947 rank 27 total_num 1420 1429
checkcorrect (5111, 2, 9881) (5111, 2, 9881) real score 0.6477426081895828 Hits@1 0.09212376933895922 Hits@3 0.24542897327707455 Hits@10 0.519690576652602 MRR 0.22918774258351257 rank 9 total_num 1421 1429
checkcorrect (9256, 4, 9655) (9256, 4, 9655) real score 0.2172418087720871 Hits@1 0.09205903021784961 Hits@3 0.24525650035137034 Hits@10 0.5193253689388616 MRR 0.2290462036

#### Fine tuned

In [34]:
#function to build the big batche for path-based training
def build_big_batches_path(lower_bd, upper_bd, data, one_hop, s_t_r,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    #the set of all initial relations
    ini_r_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
        
        if i % 2 == 0: #initial relation id is always an even number
            ini_r_id_set.add(i)
    
    num_r = len(id2relation)
    num_ini_r = len(ini_r_id_set)
    
    if num_ini_r != int(num_r/2):
        raise ValueError('error when generating id2relation')
    
    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)
    random.shuffle(existing_ids)
    
    count = 0
    for s in existing_ids:
        
        #impliment the path finding algorithm to find paths between s and t
        result, length_dict = Class_2.obtain_paths('direct_neighbour', s, 'nb', lower_bd, upper_bd, one_hop)
        
        for iteration in range(2):

            #proceed only if at least three paths are between s and t
            for t in result:

                if len(s_t_r[(s,t)]) == 0:

                    raise ValueError(s,t,id2entity[s], id2entity[t])

                #we are only interested in forward link in relation prediciton
                ini_r_list = list()

                #obtain initial relations between s and t
                for r in s_t_r[(s,t)]:
                    if r % 2 == 0:#initial relation id is always an even number
                        ini_r_list.append(r)

                #if there exist more than three paths between s and t, 
                #and inital connection between s and t exists,
                #and not every r in the relation dictionary exists between s and t (although this is rare)
                #we then proceed
                if len(result[t]) >= 3 and len(ini_r_list) > 0 and len(ini_r_list) < int(num_ini_r):

                    #obtain the list form of all the paths from s to t
                    temp_path_list = list(result[t])

                    temp_pair = random.sample(temp_path_list, 3)

                    path_1, path_2, path_3 = temp_pair[0], temp_pair[1], temp_pair[2]

                    #####positive#####################
                    #append the paths: note that we add the space holder id at the end of the shorter path
                    x_p_list['1'].append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
                    x_p_list['2'].append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
                    x_p_list['3'].append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))

                    #append relation
                    r = random.choice(ini_r_list)
                    x_r_list.append([r])
                    y_list.append(1.)

                    #####negative#####################
                    #append the paths: note that we add the space holder id at the end
                    #of the shorter path
                    x_p_list['1'].append(list(path_1) + [num_r]*abs(len(path_1)-upper_bd))
                    x_p_list['2'].append(list(path_2) + [num_r]*abs(len(path_2)-upper_bd))
                    x_p_list['3'].append(list(path_3) + [num_r]*abs(len(path_3)-upper_bd))

                    #append relation
                    neg_r_list = list(ini_r_id_set.difference(set(ini_r_list)))
                    r_ran = random.choice(neg_r_list)
                    x_r_list.append([r_ran])
                    y_list.append(0.)
        
        count += 1
        if count % 100 == 0:
            print('generating big-batches for path-based model', count, len(existing_ids))

In [35]:
#Again, it is too slow to run the path-finding algorithm again and again on the complete FB15K-237
#Instead, we will find the subgraph for each entity once.
#then in the subgraph based training, the subgraphs are stored and used for multiple times
def store_subgraph_dicts(lower_bd, upper_bd, data, one_hop, s_t_r,
                         relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
    
    num_r = len(id2relation)
    
    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
    
    #the ids to start path finding
    existing_ids = list(existing_ids)
    random.shuffle(existing_ids)
    
    #Dict stores the subgraph for each entity
    Dict_1 = dict()
    
    count = 0
    for s in existing_ids:
        
        path_set = set()
            
        result, length_dict = Class_2.obtain_paths('any_target', s, 'any', lower_bd, upper_bd, one_hop)

        for t_ in result:
            for path in result[t_]:
                path_set.add(path)

        del(result, length_dict)
        
        path_list = list(path_set)
        
        path_select = random.sample(path_list, min(len(path_list), 100))
            
        Dict_1[s] = deepcopy(path_select)
        
        count += 1
        if count % 100 == 0:
            print('generating and storing paths for the path-based model', count, len(existing_ids))
        
    return(Dict_1)

In [36]:
#function to build the big-batch for one-hope neighbor training
def build_big_batches_subgraph(lower_bd, upper_bd, data, one_hop, s_t_r,
                      x_s_list, x_t_list, x_r_list, y_list, Dict,
                      relation2id, entity2id, id2relation, id2entity):
    
    #the set of all relation IDs
    relation_id_set = set()
    
    #the set of all initial relations
    ini_r_id_set = set()
    
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
        
        if i % 2 == 0: #initial relation id is always an even number
            ini_r_id_set.add(i)
    
    num_r = len(id2relation)
    num_ini_r = len(ini_r_id_set)
    
    if num_ini_r != int(num_r/2):
        raise ValueError('error when generating id2relation')
        
    #if an entity has at least three out-stretching paths, it is a qualified one
    qualified = set()
    for e in Dict:
        if len(Dict[e]) >= 3:
            qualified.add(e)
    qualified = list(qualified)
    
    data = list(data)
    
    for iteration in range(2):

        data = shuffle(data)

        for i_0 in range(len(data)):

            triple = data[i_0]

            s, r, t = triple[0], triple[1], triple[2] #obtain entities and relation IDs

            if s in qualified and t in qualified:

                #obtain the path list for true entities
                path_s, path_t = list(Dict[s]), list(Dict[t])

                #####positive step###########
                #randomly obtain three paths for true entities
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative step for relation###########
                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                neg_r_list = list(ini_r_id_set.difference({r}))
                r_ran = random.choice(neg_r_list)
                x_r_list.append([r_ran])
                y_list.append(0.)
                
                ##############################################
                ##############################################
                #randomly choose two negative sampled entities
                s_ran = random.choice(qualified)
                t_ran = random.choice(qualified)

                #obtain the path list for random entities
                path_s_ran, path_t_ran = list(Dict[s_ran]), list(Dict[t_ran])
                
                #####positive step#################
                #Again: randomly obtain three paths
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative for source entity###########
                #randomly obtain three paths
                temp_s = random.sample(path_s_ran, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(0.)

                #####positive step###########
                #Again: randomly obtain three paths
                temp_s = random.sample(path_s, 3)
                temp_t = random.sample(path_t, 3)
                s_p_1, s_p_2, s_p_3 = temp_s[0], temp_s[1], temp_s[2]
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(1.)

                #####negative for target entity###########
                #randomly obtain three paths
                temp_t = random.sample(path_t_ran, 3)
                t_p_1, t_p_2, t_p_3 = temp_t[0], temp_t[1], temp_t[2]

                #append the paths: note that we add the space holder id at the end of the shorter path
                x_s_list['1'].append(list(s_p_1) + [num_r]*abs(len(s_p_1)-upper_bd))
                x_s_list['2'].append(list(s_p_2) + [num_r]*abs(len(s_p_2)-upper_bd))
                x_s_list['3'].append(list(s_p_3) + [num_r]*abs(len(s_p_3)-upper_bd))

                x_t_list['1'].append(list(t_p_1) + [num_r]*abs(len(t_p_1)-upper_bd))
                x_t_list['2'].append(list(t_p_2) + [num_r]*abs(len(t_p_2)-upper_bd))
                x_t_list['3'].append(list(t_p_3) + [num_r]*abs(len(t_p_3)-upper_bd))

                #append relation
                x_r_list.append([r])
                y_list.append(0.)

            if i_0 % 200 == 0:
                print('generating big-batches for subgraph-based model', i_0, len(data), iteration)

In [37]:
###fine tune the path-based model
lower_bd = lower_bound
upper_bd = upper_bound_path
batch_size = 32

#define the training lists
train_p_list, train_r_list, train_y_list = {'1': [], '2': [], '3': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches_path(lower_bd, upper_bd, data_ind, one_hop_ind, s_t_r_ind,
                      train_p_list, train_r_list, train_y_list,
                      relation2id, entity2id, id2relation, id2entity)   

#######################################
###do the training#####################

#generate the input arrays
x_train_1 = np.asarray(train_p_list['1'], dtype='int')
x_train_2 = np.asarray(train_p_list['2'], dtype='int')
x_train_3 = np.asarray(train_p_list['3'], dtype='int')
x_train_r = np.asarray(train_r_list, dtype='int')
y_train = np.asarray(train_y_list, dtype='int')

model.fit([x_train_1, x_train_2, x_train_3, x_train_r], y_train,
           batch_size=batch_size, epochs=5)

generating big-batches for path-based model 100 922
generating big-batches for path-based model 200 922
generating big-batches for path-based model 300 922
generating big-batches for path-based model 400 922
generating big-batches for path-based model 500 922
generating big-batches for path-based model 600 922
generating big-batches for path-based model 700 922
generating big-batches for path-based model 800 922
generating big-batches for path-based model 900 922
Epoch 1/2
110/110 [==============================] - 16s 73ms/step - loss: 0.2073 - binary_accuracy: 0.9459
Epoch 2/2
110/110 [==============================] - 8s 73ms/step - loss: 0.1328 - binary_accuracy: 0.9648


In [38]:
###fine tune the subgraph model
lower_bd = lower_bound
upper_bd = upper_bound_subg
batch_size = 32

Dict_train_ind = store_subgraph_dicts(lower_bd, upper_bd, data_ind, one_hop_ind, s_t_r_ind,
                         relation2id, entity2id, id2relation, id2entity)

#define the training lists
train_s_list, train_t_list, train_r_list, train_y_list = {'1': [], '2': [], '3': []}, {'1': [], '2': [], '3': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches_subgraph(lower_bd, upper_bd, data_ind, one_hop_ind, s_t_r_ind,
                      train_s_list, train_t_list, train_r_list, train_y_list, Dict_train_ind,
                      relation2id, entity2id, id2relation, id2entity)

#######################################
###do the training#####################

#generate the input arrays
x_train_s_1 = np.asarray(train_s_list['1'], dtype='int')
x_train_s_2 = np.asarray(train_s_list['2'], dtype='int')
x_train_s_3 = np.asarray(train_s_list['3'], dtype='int')

x_train_t_1 = np.asarray(train_t_list['1'], dtype='int')
x_train_t_2 = np.asarray(train_t_list['2'], dtype='int')
x_train_t_3 = np.asarray(train_t_list['3'], dtype='int')

x_train_r = np.asarray(train_r_list, dtype='int')
y_train = np.asarray(train_y_list, dtype='int')

model_2.fit([x_train_s_1, x_train_s_2, x_train_s_3, 
             x_train_t_1, x_train_t_2, x_train_t_3, x_train_r], y_train,
             batch_size=batch_size, epochs=5)

generating and storing paths for the path-based model 100 922
generating and storing paths for the path-based model 200 922
generating and storing paths for the path-based model 300 922
generating and storing paths for the path-based model 400 922
generating and storing paths for the path-based model 500 922
generating and storing paths for the path-based model 600 922
generating and storing paths for the path-based model 700 922
generating and storing paths for the path-based model 800 922
generating and storing paths for the path-based model 900 922
generating big-batches for subgraph-based model 0 1618 0
generating big-batches for subgraph-based model 200 1618 0
generating big-batches for subgraph-based model 400 1618 0
generating big-batches for subgraph-based model 600 1618 0
generating big-batches for subgraph-based model 800 1618 0
generating big-batches for subgraph-based model 1000 1618 0
generating big-batches for subgraph-based model 1200 1618 0
generating big-batches for su

In [39]:
########################################################
#obtain the Hits@N for relation prediction##############

#we select all the triples in the inductive test set
selected = list(data_ind_test)

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0
Hits_at_3 = 0
Hits_at_10 = 0
MRR_raw = 0.

for i in range(len(selected)):
    
    s_true, r_true, t_true = selected[i][0], selected[i][1], selected[i][2]
    
    #run the path-based scoring
    score_dict_path = path_based_relation_scoring(s_true, t_true, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)
    
    #run the one-hop neighbour based scoring
    score_dict_subg = subgraph_relation_scoring(s_true, t_true, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #final score dict
    score_dict = defaultdict(float)
    
    for r in score_dict_path:
        score_dict[r] += score_dict_path[r]
    for r in score_dict_subg:
        score_dict[r] += score_dict_subg[r]
    
    #[... [score, r], ...]
    temp_list = list()
    
    for r in id2relation:
        
        #again, we only care about initial relation prediciton
        if r % 2 == 0:
        
            if r in score_dict:

                temp_list.append([score_dict[r], r])

            else:

                temp_list.append([0.0, r])
        
    sorted_list = sorted(temp_list, key = lambda x: x[0], reverse=True)
    
    p = 0
    exist_tri = 0
    
    while p < len(sorted_list) and sorted_list[p][1] != r_true:
        
        #moreover, we want to remove existing triples
        if ((s_true, sorted_list[p][1], t_true) in data_test) or (
            (s_true, sorted_list[p][1], t_true) in data_valid) or (
            (s_true, sorted_list[p][1], t_true) in data) or (
            (s_true, sorted_list[p][1], t_true) in data_ind) or (
            (s_true, sorted_list[p][1], t_true) in data_ind_valid) or (
            (s_true, sorted_list[p][1], t_true) in data_ind_test):
            
            exist_tri += 1
            
        p += 1
    
    if p - exist_tri == 0:
        
        Hits_at_1 += 1
        
    if p - exist_tri < 3:
        
        Hits_at_3 += 1
        
    if p - exist_tri < 10:
        
        Hits_at_10 += 1
        
    MRR_raw += 1./float(p - exist_tri + 1.) 
        
    print('checkcorrect', r_true, sorted_list[p][1],
          'real score', sorted_list[p][0],
          'Hits@1', Hits_at_1/(i+1),
          'Hits@3', Hits_at_3/(i+1),
          'Hits@10', Hits_at_10/(i+1),
          'MRR', MRR_raw/(i+1),
          'cur_rank', p - exist_tri,
          'abs_cur_rank', p,
          'total_num', i, len(selected))

0 0
9 10 10
checkcorrect 0 0 real score 0.3973050579428673 Hits@1 0.0 Hits@3 1.0 Hits@10 1.0 MRR 0.5 cur_rank 1 abs_cur_rank 1 total_num 0 188
0 1
9 11 31
checkcorrect 0 0 real score 0.7147584974765777 Hits@1 0.5 Hits@3 1.0 Hits@10 1.0 MRR 0.75 cur_rank 0 abs_cur_rank 0 total_num 1 188
0 1
9 34 10
checkcorrect 0 0 real score 0.7171109318733215 Hits@1 0.6666666666666666 Hits@3 1.0 Hits@10 1.0 MRR 0.8333333333333334 cur_rank 0 abs_cur_rank 0 total_num 2 188
9 61
9 13 22
checkcorrect 0 0 real score 1.6940622508525849 Hits@1 0.75 Hits@3 1.0 Hits@10 1.0 MRR 0.875 cur_rank 0 abs_cur_rank 0 total_num 3 188
9 11
9 47 31
checkcorrect 0 0 real score 1.4414083257317545 Hits@1 0.8 Hits@3 1.0 Hits@10 1.0 MRR 0.9 cur_rank 0 abs_cur_rank 0 total_num 4 188
0 2
9 14 58
checkcorrect 8 8 real score 0.2447445958852768 Hits@1 0.6666666666666666 Hits@3 1.0 Hits@10 1.0 MRR 0.8333333333333334 cur_rank 1 abs_cur_rank 1 total_num 5 188
9 30
9 22 9
checkcorrect 8 8 real score 0.5786192186176777 Hits@1 0.57142857

9 38
9 18 8
checkcorrect 0 0 real score 1.2535340383648874 Hits@1 0.7843137254901961 Hits@3 0.9803921568627451 Hits@10 1.0 MRR 0.8830065359477124 cur_rank 0 abs_cur_rank 0 total_num 50 188
9 54
9 23 20
checkcorrect 0 0 real score 1.7500863373279572 Hits@1 0.7884615384615384 Hits@3 0.9807692307692307 Hits@10 1.0 MRR 0.8852564102564102 cur_rank 0 abs_cur_rank 0 total_num 51 188
9 5
9 71 28
checkcorrect 0 0 real score 1.5344414383172988 Hits@1 0.7924528301886793 Hits@3 0.9811320754716981 Hits@10 1.0 MRR 0.8874213836477987 cur_rank 0 abs_cur_rank 0 total_num 52 188
9 53
9 35 13
checkcorrect 0 0 real score 1.4713187865912913 Hits@1 0.7962962962962963 Hits@3 0.9814814814814815 Hits@10 1.0 MRR 0.8895061728395062 cur_rank 0 abs_cur_rank 0 total_num 53 188
9 4
9 23 22
checkcorrect 10 10 real score 1.7709940746426582 Hits@1 0.8 Hits@3 0.9818181818181818 Hits@10 1.0 MRR 0.8915151515151515 cur_rank 0 abs_cur_rank 0 total_num 54 188
9 86
9 32 19
checkcorrect 0 0 real score 1.4118973553180694 Hits@1

9 150
9 12 13
checkcorrect 0 0 real score 1.5491866767406464 Hits@1 0.8526315789473684 Hits@3 0.9789473684210527 Hits@10 1.0 MRR 0.9164912280701754 cur_rank 0 abs_cur_rank 0 total_num 94 188
9 7
9 26 9
checkcorrect 0 0 real score 1.6799812585115432 Hits@1 0.8541666666666666 Hits@3 0.9791666666666666 Hits@10 1.0 MRR 0.9173611111111111 cur_rank 0 abs_cur_rank 0 total_num 95 188
0 0
0 28 1
checkcorrect 0 0 real score 0.0 Hits@1 0.8556701030927835 Hits@3 0.979381443298969 Hits@10 1.0 MRR 0.9182130584192439 cur_rank 0 abs_cur_rank 0 total_num 96 188
0 0
9 32 5
checkcorrect 8 8 real score 0.2664215698838234 Hits@1 0.8469387755102041 Hits@3 0.9795918367346939 Hits@10 1.0 MRR 0.9139455782312925 cur_rank 1 abs_cur_rank 1 total_num 97 188
9 65
9 27 34
checkcorrect 0 0 real score 1.5753029331564905 Hits@1 0.8484848484848485 Hits@3 0.9797979797979798 Hits@10 1.0 MRR 0.9148148148148147 cur_rank 0 abs_cur_rank 0 total_num 98 188
0 1
9 15 46
checkcorrect 0 0 real score 0.6504505783319473 Hits@1 0.85 

9 45
9 28 37
checkcorrect 0 0 real score 1.6402869790792465 Hits@1 0.8561151079136691 Hits@3 0.9784172661870504 Hits@10 1.0 MRR 0.9189448441247002 cur_rank 0 abs_cur_rank 0 total_num 138 188
0 1
9 45 22
checkcorrect 0 0 real score 0.6212806850671768 Hits@1 0.8571428571428571 Hits@3 0.9785714285714285 Hits@10 1.0 MRR 0.9195238095238096 cur_rank 0 abs_cur_rank 0 total_num 139 188
9 93
9 31 26
checkcorrect 0 0 real score 1.6512427419424056 Hits@1 0.8581560283687943 Hits@3 0.9787234042553191 Hits@10 1.0 MRR 0.9200945626477542 cur_rank 0 abs_cur_rank 0 total_num 140 188
9 87
9 39 44
checkcorrect 0 0 real score 1.5928798407316207 Hits@1 0.8591549295774648 Hits@3 0.9788732394366197 Hits@10 1.0 MRR 0.9206572769953053 cur_rank 0 abs_cur_rank 0 total_num 141 188
0 2
9 14 9
checkcorrect 0 0 real score 0.46829487979412077 Hits@1 0.8531468531468531 Hits@3 0.9790209790209791 Hits@10 1.0 MRR 0.9177156177156178 cur_rank 1 abs_cur_rank 1 total_num 142 188
9 132
9 58 37
checkcorrect 0 0 real score 1.661

9 16 34
checkcorrect 0 0 real score 0.6142259627580643 Hits@1 0.8524590163934426 Hits@3 0.9726775956284153 Hits@10 1.0 MRR 0.9163023679417123 cur_rank 0 abs_cur_rank 0 total_num 182 188
9 4
9 22 19
checkcorrect 10 10 real score 1.9311034083366394 Hits@1 0.8532608695652174 Hits@3 0.9728260869565217 Hits@10 1.0 MRR 0.9167572463768117 cur_rank 0 abs_cur_rank 0 total_num 183 188
9 22
9 50 58
checkcorrect 0 0 real score 1.5718801110982894 Hits@1 0.8540540540540541 Hits@3 0.972972972972973 Hits@10 1.0 MRR 0.9172072072072073 cur_rank 0 abs_cur_rank 0 total_num 184 188
9 18
9 16 10
checkcorrect 0 0 real score 1.3778696570545435 Hits@1 0.8548387096774194 Hits@3 0.9731182795698925 Hits@10 1.0 MRR 0.9176523297491039 cur_rank 0 abs_cur_rank 0 total_num 185 188
9 14
9 12 33
checkcorrect 0 0 real score 1.2510857969522475 Hits@1 0.8556149732620321 Hits@3 0.9732620320855615 Hits@10 1.0 MRR 0.9180926916221034 cur_rank 0 abs_cur_rank 0 total_num 186 188
0 0
0 1 51
checkcorrect 0 0 real score 0.0 Hits@1 

In [40]:
###########################################
##obtain the AUC-PR for the test triples###
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
import matplotlib.pyplot as plt

#we select all the triples in the inductive test set
pos_triples = list(data_ind_test)

#we build the negative samples by randomly replace head or tail entity in the triple.
neg_triples = list()

for i in range(len(pos_triples)):
    
    s_pos, r_pos, t_pos = pos_triples[i][0], pos_triples[i][1], pos_triples[i][2]
    
    #decide to replace the head or tail entity
    number_0 = random.uniform(0, 1)
    
    if number_0 < 0.5: #replace head entity
        
        s_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_neg, r_pos, t_pos) in data_test) or (
               (s_neg, r_pos, t_pos) in data_valid) or (
               (s_neg, r_pos, t_pos) in data) or (
               (s_neg, r_pos, t_pos) in data_ind) or (
               (s_neg, r_pos, t_pos) in data_ind_valid) or (
               (s_neg, r_pos, t_pos) in data_ind_test):
            
            s_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_neg, r_pos, t_pos))
    
    else: #replace tail entity

        t_neg = random.choice(list(new_ent_set))
        
        #filter out the existing triples
        while ((s_pos, r_pos, t_neg) in data_test) or (
               (s_pos, r_pos, t_neg) in data_valid) or (
               (s_pos, r_pos, t_neg) in data) or (
               (s_pos, r_pos, t_neg) in data_ind) or (
               (s_pos, r_pos, t_neg) in data_ind_valid) or (
               (s_pos, r_pos, t_neg) in data_ind_test):
            
            t_neg = random.choice(list(new_ent_set))
        
        neg_triples.append((s_pos, r_pos, t_neg))

if len(pos_triples) != len(neg_triples):
    raise ValueError('error when generating negative triples')
        
#combine all triples
all_triples = pos_triples + neg_triples

#obtain the label array
arr1 = np.ones((len(pos_triples),))
arr2 = np.zeros((len(neg_triples),))
y_test = np.concatenate((arr1, arr2))

#shuffle positive and negative triples (optional)
all_triples, y_test = shuffle(all_triples, y_test)

#obtain the score aray
y_score = np.zeros((len(y_test),))

#implement the scoring
for i in range(len(all_triples)):
    
    s, r, t = all_triples[i][0], all_triples[i][1], all_triples[i][2]
    
    #path_score = path_based_triple_scoring(s, r, t, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)
    
    subg_score = subgraph_triple_scoring(s, r, t, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #ave_score = (path_score + subg_score)/float(2)
    
    #y_score[i] = ave_score
    y_score[i] = subg_score
    
    if i % 20 == 0 and i > 0:
        print('evaluating scores', i, len(all_triples))
        
        # Data to plot precision - recall curve
        precision, recall, thresholds = precision_recall_curve(y_test[:i], y_score[:i])
        # Use AUC function to calculate the area under the curve of precision recall curve
        auc_precision_recall = auc(recall, precision)
        print('AUC-PR is:', auc_precision_recall)
        
        
# Data to plot precision - recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_score)
# Use AUC function to calculate the area under the curve of precision recall curve
auc_precision_recall = auc(recall, precision)
print('AUC-PR is:', auc_precision_recall)

evaluating scores 20 376
AUC-PR is: 0.8158124039702987
evaluating scores 40 376
AUC-PR is: 0.770603450876653
evaluating scores 60 376
AUC-PR is: 0.7587513046465775
evaluating scores 80 376
AUC-PR is: 0.7889110246028787
evaluating scores 100 376
AUC-PR is: 0.77768457943568
evaluating scores 120 376
AUC-PR is: 0.7778079025696366
evaluating scores 140 376
AUC-PR is: 0.7901361514723642
evaluating scores 160 376
AUC-PR is: 0.7539759441124845
evaluating scores 180 376
AUC-PR is: 0.7480333614245009
evaluating scores 200 376
AUC-PR is: 0.7373484679242981
evaluating scores 220 376
AUC-PR is: 0.7403965974871085
evaluating scores 240 376
AUC-PR is: 0.7600854965707611
evaluating scores 260 376
AUC-PR is: 0.7360974188142251
evaluating scores 280 376
AUC-PR is: 0.7377639545950183
evaluating scores 300 376
AUC-PR is: 0.725380399187881
evaluating scores 320 376
AUC-PR is: 0.7198041945682027
evaluating scores 340 376
AUC-PR is: 0.7173594115404267
evaluating scores 360 376
AUC-PR is: 0.7250464293392811


In [ ]:
######################################################
#obtain the Hits@N for entity prediction##############

#we select all the triples in the inductive test set
selected = list(data_ind_test)

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0
Hits_at_3 = 0
Hits_at_10 = 0
MRR_raw = 0.

for i in range(len(selected)):
    
    triple_list = list()
    
    #score the true triple
    s_pos, r_pos, t_pos = selected[i][0], selected[i][1], selected[i][2]

    #path_score = path_based_triple_scoring(s_pos, r_pos, t_pos, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

    subg_score = subgraph_triple_scoring(s_pos, r_pos, t_pos, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)
    
    #ave_score = (path_score + subg_score)/float(2)
    
    triple_list.append([(s_pos, r_pos, t_pos), subg_score])
    
    #generate the 50 random samples
    for sub_i in range(50):
        
        #decide to replace the head or tail entity
        number_0 = random.uniform(0, 1)

        if number_0 < 0.5: #replace head entity
            
            s_neg = random.choice(list(new_ent_set))
            
            while ((s_neg, r_pos, t_pos) in data_test) or (
                   (s_neg, r_pos, t_pos) in data_valid) or (
                   (s_neg, r_pos, t_pos) in data) or (
                   (s_neg, r_pos, t_pos) in data_ind) or (
                   (s_neg, r_pos, t_pos) in data_ind_valid) or (
                   (s_neg, r_pos, t_pos) in data_ind_test):

                s_neg = random.choice(list(new_ent_set))
            
            #path_score = path_based_triple_scoring(s_neg, r_pos, t_pos, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

            subg_score = subgraph_triple_scoring(s_neg, r_pos, t_pos, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)

            #ave_score = (path_score + subg_score)/float(2)

            triple_list.append([(s_neg, r_pos, t_pos), subg_score])
            
        else: #replace tail entity

            t_neg = random.choice(list(new_ent_set))
            
            #filter out the existing triples
            while ((s_pos, r_pos, t_neg) in data_test) or (
                   (s_pos, r_pos, t_neg) in data_valid) or (
                   (s_pos, r_pos, t_neg) in data) or (
                   (s_pos, r_pos, t_neg) in data_ind) or (
                   (s_pos, r_pos, t_neg) in data_ind_valid) or (
                   (s_pos, r_pos, t_neg) in data_ind_test):

                t_neg = random.choice(list(new_ent_set))
            
            #path_score = path_based_triple_scoring(s_pos, r_pos, t_neg, lower_bound, upper_bound_path, one_hop_ind, id2relation, model)

            subg_score = subgraph_triple_scoring(s_pos, r_pos, t_neg, lower_bound, upper_bound_subg, one_hop_ind, id2relation, model_2)

            #ave_score = (path_score + subg_score)/float(2)

            triple_list.append([(s_pos, r_pos, t_neg), subg_score])
            
    #random shuffle!
    random.shuffle(triple_list)
    
    #sort
    sorted_list = sorted(triple_list, key = lambda x: x[-1], reverse=True)
    
    p = 0
    
    while p < len(sorted_list) and sorted_list[p][0] != (s_pos, r_pos, t_pos):
            
        p += 1
    
    if p == 0:
        
        Hits_at_1 += 1
        
    if p < 3:
        
        Hits_at_3 += 1
        
    if p < 10:
        
        Hits_at_10 += 1
        
    MRR_raw += 1./float(p + 1.) 
        
    print('checkcorrect', (s_pos, r_pos, t_pos), sorted_list[p][0],
          'real score', sorted_list[p][-1],
          'Hits@1', Hits_at_1/(i+1),
          'Hits@3', Hits_at_3/(i+1),
          'Hits@10', Hits_at_10/(i+1),
          'MRR', MRR_raw/(i+1),
          'rank', p,
          'total_num', i, len(selected))

checkcorrect (3616, 0, 3274) (3616, 0, 3274) real score 0.4064545691013336 Hits@1 0.0 Hits@3 0.0 Hits@10 0.0 MRR 0.03571428571428571 rank 27 total_num 0 188
checkcorrect (3613, 0, 3144) (3613, 0, 3144) real score 0.7161331176757812 Hits@1 0.0 Hits@3 0.5 Hits@10 0.5 MRR 0.26785714285714285 rank 1 total_num 1 188
checkcorrect (2799, 0, 3251) (2799, 0, 3251) real score 0.7103965640068054 Hits@1 0.0 Hits@3 0.3333333333333333 Hits@10 0.6666666666666666 MRR 0.2261904761904762 rank 6 total_num 2 188
checkcorrect (3170, 0, 2776) (3170, 0, 2776) real score 0.7012916684150696 Hits@1 0.0 Hits@3 0.25 Hits@10 0.75 MRR 0.20535714285714285 rank 6 total_num 3 188
checkcorrect (3074, 0, 3194) (3074, 0, 3194) real score 0.749379575252533 Hits@1 0.0 Hits@3 0.4 Hits@10 0.8 MRR 0.23095238095238094 rank 2 total_num 4 188
checkcorrect (3329, 8, 2803) (3329, 8, 2803) real score 0.287155294418335 Hits@1 0.0 Hits@3 0.3333333333333333 Hits@10 0.6666666666666666 MRR 0.19912698412698412 rank 24 total_num 5 188
che

checkcorrect (2940, 0, 3343) (2940, 0, 3343) real score 0.7114482671022415 Hits@1 0.0 Hits@3 0.3404255319148936 Hits@10 0.48936170212765956 MRR 0.1805547654622057 rank 3 total_num 46 188
checkcorrect (3339, 0, 3170) (3339, 0, 3170) real score 0.5601408511400223 Hits@1 0.0 Hits@3 0.3333333333333333 Hits@10 0.4791666666666667 MRR 0.17839577195097386 rank 12 total_num 47 188
checkcorrect (3212, 8, 3283) (3212, 8, 3283) real score 0.5529832601547241 Hits@1 0.0 Hits@3 0.32653061224489793 Hits@10 0.46938775510204084 MRR 0.1764557221832669 rank 11 total_num 48 188
checkcorrect (3377, 8, 3576) (3377, 8, 3576) real score 0.0 Hits@1 0.0 Hits@3 0.32 Hits@10 0.46 MRR 0.17345292352907526 rank 37 total_num 49 188
checkcorrect (3154, 0, 3597) (3154, 0, 3597) real score 0.25739180445671084 Hits@1 0.0 Hits@3 0.3137254901960784 Hits@10 0.45098039215686275 MRR 0.17058182751923118 rank 36 total_num 50 188
checkcorrect (3150, 0, 3219) (3150, 0, 3219) real score 0.726323926448822 Hits@1 0.0 Hits@3 0.3076923

checkcorrect (3572, 0, 2813) (3572, 0, 2813) real score 0.5788687020540237 Hits@1 0.02247191011235955 Hits@3 0.24719101123595505 Hits@10 0.4044943820224719 MRR 0.17062255277834687 rank 23 total_num 88 188
checkcorrect (3540, 0, 2833) (3540, 0, 2833) real score 0.6214325815439224 Hits@1 0.022222222222222223 Hits@3 0.24444444444444444 Hits@10 0.4111111111111111 MRR 0.17150452441414302 rank 3 total_num 89 188
checkcorrect (2859, 0, 3040) (2859, 0, 3040) real score 0.6616534888744354 Hits@1 0.02197802197802198 Hits@3 0.24175824175824176 Hits@10 0.4175824175824176 MRR 0.170718760409592 rank 9 total_num 90 188
checkcorrect (2956, 0, 2903) (2956, 0, 2903) real score 0.7293300211429596 Hits@1 0.021739130434782608 Hits@3 0.25 Hits@10 0.42391304347826086 MRR 0.17248631011528484 rank 2 total_num 91 188
checkcorrect (3388, 0, 2809) (3388, 0, 2809) real score 0.7448042243719101 Hits@1 0.03225806451612903 Hits@3 0.25806451612903225 Hits@10 0.43010752688172044 MRR 0.18138430678071185 rank 0 total_num